# Modelo de programación lineal

In [1]:
import pandas as pd
import gurobipy as grb

## Entrada de datos: Instancia 4x200x1.5x1

In [2]:
datafile="4x200x1.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

## Modelo

In [3]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Academic license - for non-commercial use only - expires 2021-06-11
Using license file C:\Users\elPatan03\gurobi.lic
Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x0e215afb
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 18.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 14s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10375955 rows and 457944 columns (presolve time = 24s) ...
Presolve removed 10375955 r

  2327  2034  185.64690   19  481  184.00000  185.64690  0.90%   696  840s

Cutting planes:
  Learned: 5
  Gomory: 36
  Cover: 130
  Projected implied bound: 2
  Clique: 586
  MIR: 98
  StrongCG: 71
  Flow cover: 117
  GUB cover: 26
  Zero half: 217
  RLT: 2
  Relax-and-lift: 21

Explored 2327 nodes (2136327 simplex iterations) in 847.74 seconds
Thread count was 6 (of 6 available processors)

Solution count 7: 184 183 181 ... -0

Optimal solution found (tolerance 1.00e-04)
Best objective 1.840000000000e+02, best bound 1.840000000000e+02, gap 0.0000%


## Entrada de datos: Instancia 4x200x1.5x2

In [4]:
datafile="4x200x1.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [5]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0xb1be0f4c
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10396739 rows and 458844 columns (presolve time = 24s) ...
Presolve removed 10396739 rows and 458844 columns (presolve time = 29s) ...
Presolve removed 10439819 rows and 458844 columns (presolve time = 3

 24232 15227 infeasible  149       185.00000  186.00000  0.54%   122  465s
 25318 15228  186.00000   93   97  185.00000  186.00000  0.54%   121  474s
 25320 15229  186.00000   69  102  185.00000  186.00000  0.54%   121  475s
 25323 15231  186.00000  132  121  185.00000  186.00000  0.54%   121  480s
 25329 15235  186.00000   81  101  185.00000  186.00000  0.54%   121  486s
 25333 15238  186.00000   78   98  185.00000  186.00000  0.54%   121  490s
 25339 15242  186.00000   26  118  185.00000  186.00000  0.54%   121  495s
 25342 15249  186.00000   23  101  185.00000  186.00000  0.54%   126  500s
 25352 15258  186.00000   25  193  185.00000  186.00000  0.54%   126  505s
 25370 15270  186.00000   26  214  185.00000  186.00000  0.54%   126  513s
 25376 15276  186.00000   27  152  185.00000  186.00000  0.54%   126  515s
 25398 15291  186.00000   28  171  185.00000  186.00000  0.54%   126  520s
 25407 15297  186.00000   29   99  185.00000  186.00000  0.54%   126  530s
 25416 15358  186.00000  

 143766 42340  186.00000  166   74  185.00000  186.00000  0.54%   135 1408s
 147472 42614 infeasible  176       185.00000  186.00000  0.54%   136 1421s
 148193 43115  186.00000  156   64  185.00000  186.00000  0.54%   136 1438s
 149569 43463  186.00000  180   55  185.00000  186.00000  0.54%   136 1447s
 150690 44083 infeasible  181       185.00000  186.00000  0.54%   136 1455s
 152434 44957  186.00000  170   51  185.00000  186.00000  0.54%   136 1462s
 154289 45230  186.00000  159  102  185.00000  186.00000  0.54%   135 1468s
 154882 45639 infeasible  146       185.00000  186.00000  0.54%   135 1475s
 156341 45691 infeasible  146       185.00000  186.00000  0.54%   135 1483s
 156932 46534  186.00000  138   86  185.00000  186.00000  0.54%   136 1498s
 159603 46637 infeasible  156       185.00000  186.00000  0.54%   136 1509s
 159854 47149  186.00000  152   84  185.00000  186.00000  0.54%   136 1516s
 161358 47364  186.00000  175  129  185.00000  186.00000  0.54%   136 1529s
 161823 4773

 280291 88770 infeasible  210       185.00000  186.00000  0.54%   139 2348s
 281282 89087 infeasible  195       185.00000  186.00000  0.54%   139 2354s
 282293 89420 infeasible  214       185.00000  186.00000  0.54%   139 2360s
 283396 89860  186.00000  193   55  185.00000  186.00000  0.54%   139 2366s
 284453 90261 infeasible  197       185.00000  186.00000  0.54%   139 2372s
 285484 90618 infeasible  207       185.00000  186.00000  0.54%   139 2381s
 286822 90952  186.00000  217   56  185.00000  186.00000  0.54%   139 2387s
 287912 91315  186.00000  212   61  185.00000  186.00000  0.54%   139 2400s
 288956 91531  186.00000  196   44  185.00000  186.00000  0.54%   140 2407s
 289570 91977 infeasible  206       185.00000  186.00000  0.54%   140 2415s
 290820 92038 infeasible  212       185.00000  186.00000  0.54%   140 2420s
 291267 92555 infeasible  193       185.00000  186.00000  0.54%   140 2427s
 292652 92798  186.00000  191   81  185.00000  186.00000  0.54%   140 2436s
 293283 9311

 382101 127573  186.00000  158   97  185.00000  186.00000  0.54%   143 3142s
 383287 128031  186.00000  177   45  185.00000  186.00000  0.54%   143 3148s
 384475 128421  186.00000  169   59  185.00000  186.00000  0.54%   143 3155s
 385700 128943 infeasible  159       185.00000  186.00000  0.54%   143 3160s
 386774 129473  186.00000  163   41  185.00000  186.00000  0.54%   143 3166s
 387764 129958  186.00000  167   84  185.00000  186.00000  0.54%   143 3171s
 388670 130376  186.00000  174   70  185.00000  186.00000  0.54%   143 3180s
 389787 130372  186.00000  166   74  185.00000  186.00000  0.54%   143 3197s
 389849 130895  186.00000  166   76  185.00000  186.00000  0.54%   143 3204s
 391150 131325 infeasible  177       185.00000  186.00000  0.54%   143 3210s
 392255 131492 infeasible  173       185.00000  186.00000  0.54%   143 3221s
 392798 132080  186.00000  166   86  185.00000  186.00000  0.54%   143 3228s
 394086 132512 infeasible  163       185.00000  186.00000  0.54%   143 3234s

## Entrada de datos: Instancia 4x200x1.5x3

In [6]:
datafile="4x200x1.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [7]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x90a0c7ae
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10339368 rows and 457344 columns (presolve time = 25s) ...
Presolve removed 10339368 rows and 457344 columns (presolve time = 30s) ...
Presolve removed 10376472 rows and 457344 columns (presolve time = 3

 38676 29769  177.00000  175   73  176.00000  177.00000  0.57%  98.9  461s
 39976 32406 infeasible  165       176.00000  177.00000  0.57%  98.6  470s
 43709 34637  177.00000  181   53  176.00000  177.00000  0.57%  96.3  475s
 44815 34748 infeasible  236       176.00000  177.00000  0.57%  95.2  483s
 44958 35480  177.00000  306   47  176.00000  177.00000  0.57%  95.1  485s
 47957 38233  177.00000  253   36  176.00000  177.00000  0.57%  93.8  491s
 50546 40200  177.00000  296   38  176.00000  177.00000  0.57%  92.3  495s
 52368 40839  177.00000  160   67  176.00000  177.00000  0.57%  92.0  503s
 52376 41610  177.00000  162   63  176.00000  177.00000  0.57%  92.0  505s
 53279 43023  177.00000  234   62  176.00000  177.00000  0.57%  91.5  510s
 56075 44710  177.00000  202  114  176.00000  177.00000  0.57%  90.8  515s
 58301 46482  177.00000  194   48  176.00000  177.00000  0.57%  91.0  521s
 60072 48110  177.00000  162   93  176.00000  177.00000  0.57%  90.9  526s
 61925 49519  177.00000  

 216885 178478  177.00000  194   23  176.00000  177.00000  0.57%  83.7 1036s
 219041 180250  177.00000  178   64  176.00000  177.00000  0.57%  83.7 1040s
 221477 182064  177.00000  190   42  176.00000  177.00000  0.57%  83.5 1045s
 223486 184371  177.00000  226   67  176.00000  177.00000  0.57%  83.4 1052s
 226023 186048  177.00000  188   39  176.00000  177.00000  0.57%  83.3 1055s
 228329 188055  177.00000  190   80  176.00000  177.00000  0.57%  83.1 1061s
 230366 189068  177.00000  229   42  176.00000  177.00000  0.57%  83.1 1067s
 231439 190729  177.00000  218   81  176.00000  177.00000  0.57%  83.0 1071s
 233447 192579  177.00000  253   20  176.00000  177.00000  0.57%  82.9 1075s
 235871 193633 infeasible  332       176.00000  177.00000  0.57%  82.8 1080s
 236791 195182  177.00000  141   17  176.00000  177.00000  0.57%  82.7 1086s
 239547 196840  177.00000  185   36  176.00000  177.00000  0.57%  82.6 1094s
 239653 197414  177.00000  186   26  176.00000  177.00000  0.57%  82.6 1096s

 405330 336361  177.00000  313   26  176.00000  177.00000  0.57%  81.0 1590s
 407261 337982  177.00000  213    6  176.00000  177.00000  0.57%  80.9 1597s
 408164 338524 infeasible  289       176.00000  177.00000  0.57%  80.9 1602s
 408838 339736  177.00000  223   54  176.00000  177.00000  0.57%  80.9 1606s
 411794 342091  177.00000  203   95  176.00000  177.00000  0.57%  80.9 1612s
 414234 344077  177.00000  215   45  176.00000  177.00000  0.57%  80.9 1617s
 415495 344211  177.00000  189   65  176.00000  177.00000  0.57%  80.8 1623s
 415665 344378  177.00000  190   63  176.00000  177.00000  0.57%  80.8 1626s
 417215 346447  177.00000  255   55  176.00000  177.00000  0.57%  80.8 1632s
 418972 347709  177.00000  205   28  176.00000  177.00000  0.57%  80.8 1637s
 419727 348430  177.00000  256   70  176.00000  177.00000  0.57%  80.8 1640s
 420634 349543 infeasible  325       176.00000  177.00000  0.57%  80.8 1645s
 423879 351990  177.00000  221   28  176.00000  177.00000  0.57%  80.7 1651s

 534104 444814 infeasible  352       176.00000  177.00000  0.57%  80.1 2152s
 534635 445622  177.00000  222   44  176.00000  177.00000  0.57%  80.1 2156s
 535595 446499  177.00000  204  100  176.00000  177.00000  0.57%  80.0 2161s
 536634 447258  177.00000  290   33  176.00000  177.00000  0.57%  80.0 2165s
 537455 448034  177.00000  254   29  176.00000  177.00000  0.57%  80.0 2174s
 538411 448731 infeasible  286       176.00000  177.00000  0.57%  80.0 2180s
 539269 449736  177.00000  199   85  176.00000  177.00000  0.57%  80.1 2185s
 541166 450369  177.00000  207  102  176.00000  177.00000  0.57%  80.1 2195s
 541172 451222  177.00000  208   10  176.00000  177.00000  0.57%  80.1 2200s
 542211 452220  177.00000  206  108  176.00000  177.00000  0.57%  80.1 2207s
 543402 453272  177.00000  216  109  176.00000  177.00000  0.57%  80.1 2212s
 544586 454004  177.00000  243   23  176.00000  177.00000  0.57%  80.0 2217s
 545509 454450  177.00000  253   54  176.00000  177.00000  0.57%  80.0 2223s

 635824 530991  177.00000  203   50  176.00000  177.00000  0.57%  79.1 2791s
 636523 532263  177.00000  182   29  176.00000  177.00000  0.57%  79.1 2796s
 637894 533121  177.00000  277   63  176.00000  177.00000  0.57%  79.0 2801s
 638929 533631  177.00000  161   34  176.00000  177.00000  0.57%  79.0 2805s
 639499 534431  177.00000  215   70  176.00000  177.00000  0.57%  79.0 2811s
 640387 535259  177.00000  252   90  176.00000  177.00000  0.57%  78.9 2815s
 641411 536086  177.00000  324   68  176.00000  177.00000  0.57%  78.9 2821s
 642404 537853  177.00000  341   71  176.00000  177.00000  0.57%  78.8 2830s
 644496 538671  177.00000  311   38  176.00000  177.00000  0.57%  78.8 2838s
 645245 539215  177.00000  187   72  176.00000  177.00000  0.57%  78.8 2843s
 645900 540132  177.00000  184   28  176.00000  177.00000  0.57%  78.8 2847s
 646987 540804  177.00000  201    4  176.00000  177.00000  0.57%  78.8 2852s
 647688 542070  177.00000  279   53  176.00000  177.00000  0.57%  78.7 2858s

 735984 617280  177.00000  276   55  176.00000  177.00000  0.57%  77.2 3466s
 736682 617790  177.00000  316   38  176.00000  177.00000  0.57%  77.2 3472s
 737334 618270  177.00000  225   42  176.00000  177.00000  0.57%  77.2 3477s
 737868 618898  177.00000  193   35  176.00000  177.00000  0.57%  77.2 3483s
 738601 619579  177.00000  278   36  176.00000  177.00000  0.57%  77.2 3490s
 739406 620233  177.00000  286   38  176.00000  177.00000  0.57%  77.2 3497s
 740209 620456  177.00000  199   63  176.00000  177.00000  0.57%  77.2 3505s
 740438 621244  177.00000  250   52  176.00000  177.00000  0.57%  77.2 3513s
 741416 621861  177.00000  290   36  176.00000  177.00000  0.57%  77.2 3520s
 742181 622508  177.00000  133   77  176.00000  177.00000  0.57%  77.2 3527s
 742876 623066  177.00000  199   30  176.00000  177.00000  0.57%  77.2 3533s
 743512 623825  177.00000   95   71  176.00000  177.00000  0.57%  77.2 3540s
 744335 624491  177.00000  136   11  176.00000  177.00000  0.57%  77.2 3547s

## Entrada de datos: Instancia 4x200x1.5x4

In [8]:
datafile="4x200x1.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [9]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x5c28ce31
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10365002 rows and 457728 columns (presolve time = 24s) ...
Presolve removed 10365002 rows and 457728 columns (presolve time = 29s) ...
Presolve removed 10405326 rows and 457736 columns (presolve time = 3

  2898  2264  184.00000   29   37  182.00000  184.00000  1.10%   660  650s
  3117  3217  184.00000   40   63  182.00000  184.00000  1.10%   650  697s
  4167  3224  184.00000  159   46  182.00000  184.00000  1.10%   596  717s
  4537  3957  184.00000  200   52  182.00000  184.00000  1.10%   587  737s
  5435  4306  184.00000  241  108  182.00000  184.00000  1.10%   520  764s
  6442  4578  183.41758  410   87  182.00000  184.00000  1.10%   468  790s
  7097  5105  183.94199  296  151  182.00000  184.00000  1.10%   460  809s
  7905  5493  184.00000   50   32  182.00000  184.00000  1.10%   449  828s
  8578  6174  184.00000   90   37  182.00000  184.00000  1.10%   447  846s
  9475  6829  184.00000  158  116  182.00000  184.00000  1.10%   437  876s
 10214  7427  184.00000  212   95  182.00000  184.00000  1.10%   428  893s
 11021  8539  184.00000  269   44  182.00000  184.00000  1.10%   420  913s
 12222  9579  184.00000  121   73  182.00000  184.00000  1.10%   404  932s
 13543  9580  183.00000  

 38565  5713 infeasible  183       182.00000  183.00000  0.55%   551 2492s
 39094  5635     cutoff  168       182.00000  183.00000  0.55%   556 2518s
 39806  5399  183.00000  166  110  182.00000  183.00000  0.55%   560 2774s
 39808  5400  183.00000  169  120  182.00000  183.00000  0.55%   560 2779s
 39809  5401  183.00000  171   98  182.00000  183.00000  0.55%   560 2782s
 39810  5402  183.00000  160  278  182.00000  183.00000  0.55%   560 2786s
 39811  5402  183.00000  135   90  182.00000  183.00000  0.55%   560 2796s
 39812  5403  183.00000  166  288  182.00000  183.00000  0.55%   560 2804s
 39813  5404  183.00000  173  132  182.00000  183.00000  0.55%   560 2816s
 39814  5404  183.00000  142  255  182.00000  183.00000  0.55%   560 2822s
 39815  5405  183.00000  184  158  182.00000  183.00000  0.55%   560 2836s

Cutting planes:
  Gomory: 11
  Cover: 232
  Implied bound: 60
  Projected implied bound: 1
  Clique: 250
  MIR: 130
  StrongCG: 81
  Flow cover: 303
  GUB cover: 82
  Zero ha

## Entrada de datos: Instancia 4x200x1.5x5

In [10]:
datafile="4x200x1.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [11]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x4562fa09
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 4 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 4 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10302009 rows and 456788 columns (presolve time = 24s) ...
Presolve removed 10302009 rows and 456788 columns (presolve time = 28s) ...
Presolve removed 10349289 rows and 456804 columns (presolve time = 3

     0     0  191.12884    0  830  184.00000  191.12884  3.87%     -  381s
     0     0  191.12871    0  807  184.00000  191.12871  3.87%     -  382s
     0     0  191.11904    0  775  184.00000  191.11904  3.87%     -  387s
     0     0  191.10587    0  789  184.00000  191.10587  3.86%     -  391s
     0     0  191.09802    0  793  184.00000  191.09802  3.86%     -  394s
     0     0  191.08864    0  823  184.00000  191.08864  3.85%     -  397s
     0     0  191.08455    0  830  184.00000  191.08455  3.85%     -  400s
     0     0  191.08241    0  841  184.00000  191.08241  3.85%     -  402s
     0     0  191.08152    0  816  184.00000  191.08152  3.85%     -  403s
     0     0  191.08144    0  828  184.00000  191.08144  3.85%     -  404s
     0     0  191.06931    0  764  184.00000  191.06931  3.84%     -  408s
     0     0  191.04035    0  742  184.00000  191.04035  3.83%     -  415s
     0     0  191.03169    0  750  184.00000  191.03169  3.82%     -  417s
     0     0  191.02284  

  1882  1700  190.47899   30  837  185.00000  190.55047  3.00%  2216 2146s
  1903  1713  190.47845   32  816  185.00000  190.55047  3.00%  2242 2160s
  1923  1715  190.47425   33  816  185.00000  190.55047  3.00%  2270 2185s
  1932  1734  190.46298   34  816  185.00000  190.55047  3.00%  2279 2199s
  1954  1745  190.15857   35  818  185.00000  190.55047  3.00%  2318 2235s
  1972  1769  190.44608   36  753  185.00000  190.55047  3.00%  2356 2250s
  2002  1779  190.42711   38  758  185.00000  190.55047  3.00%  2388 2266s
  2022  1785  190.45099   40  810  185.00000  190.55047  3.00%  2418 2308s
  2035  1809  190.45007   41  821  185.00000  190.55047  3.00%  2476 2325s
  2063  1827  190.44471   43  815  185.00000  190.55047  3.00%  2511 2346s
  2090  1851  190.44426   45  752  185.00000  190.55047  3.00%  2543 2365s
  2123  1868  190.37898   47  806  185.00000  190.55047  3.00%  2581 2387s
  2151  1885  190.36413   49  783  185.00000  190.55047  3.00%  2617 2408s
  2178  1905  190.42565  

## Entrada de datos: Instancia 4x200x2x1

In [12]:
datafile="4x200x2x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [13]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x461568aa
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10538184 rows and 463460 columns (presolve time = 24s) ...
Presolve removed 10538184 rows and 463460 columns (presolve time = 29s) ...
Presolve removed 10579812 rows and 463520 columns (presolve time = 3

   761   878  160.09460  102  217  159.00000  164.26936  3.31%  2085  969s
   970  1080  164.26936    9  894  159.00000  164.26936  3.31%  1869 1006s
  1192  1233  164.09511   19  716  159.00000  164.26936  3.31%  1650 1066s
  1422  1354  163.28571   11  967  159.00000  164.26936  3.31%  1555 1135s
  1578  1355  160.48918  142  458  159.00000  164.26936  3.31%  1515 1186s
  1580  1356  162.13370  105  421  159.00000  164.26936  3.31%  1514 1191s
  1581  1357  162.75000   69  560  159.00000  164.26936  3.31%  1513 1196s
  1583  1358  160.28500  146  698  159.00000  164.26936  3.31%  1511 1206s
  1584  1359  162.75000   71  726  159.00000  164.26936  3.31%  1510 1210s
  1585  1360  163.27615   52  629  159.00000  164.26936  3.31%  1509 1217s
  1586  1360  162.66523   65  736  159.00000  164.26936  3.31%  1508 1221s
  1587  1361  164.26936    9  605  159.00000  164.26936  3.31%  1507 1227s
  1589  1362  162.82600   35  783  159.00000  164.26936  3.31%  1505 1236s
  1591  1364  164.20194  

  1720  1379  160.40346   40  837  160.00000  163.73171  2.33%  1777 1846s
  1721  1379  162.57352   71  655  160.00000  163.73171  2.33%  1776 1854s
  1722  1380  160.26723  148  786  160.00000  163.73171  2.33%  1775 1857s
  1723  1381  163.24264   48  719  160.00000  163.73171  2.33%  1774 1865s
  1725  1382  160.28500  146  712  160.00000  163.73171  2.33%  1772 1874s
  1726  1383  163.73171   23  784  160.00000  163.73171  2.33%  1771 1876s
  1727  1383  161.39914   66  706  160.00000  163.73171  2.33%  1770 1884s
  1728  1384  163.73171   13  794  160.00000  163.73171  2.33%  1769 1886s
  1729  1385  163.58250   30  624  160.00000  163.73171  2.33%  1768 1893s
  1730  1385  160.50000  140  796  160.00000  163.73171  2.33%  1767 1895s
  1731  1386  163.73171   81  692  160.00000  163.73171  2.33%  1766 1903s
  1732  1387  161.71888   84  802  160.00000  163.73171  2.33%  1765 1906s
  1733  1387  163.73171   85  658  160.00000  163.73171  2.33%  1764 1914s
  1734  1388  163.73171  

## Entrada de datos: Instancia 4x200x2x2

In [14]:
datafile="4x200x2x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [15]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x1da34643
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10545496 rows and 463664 columns (presolve time = 24s) ...
Presolve removed 10545496 rows and 463664 columns (presolve time = 29s) ...
Presolve removed 10587128 rows and 463732 columns (presolve time = 3

  1970  1657  157.00000   58  451  155.00000  158.00000  1.94%   919  696s
  1971  1658  157.00000   19  511  155.00000  158.00000  1.94%   918  701s
  1972  1659  156.91860   77  573  155.00000  158.00000  1.94%   918  710s
  1973  1659  156.00000   98  348  155.00000  158.00000  1.94%   917  717s
  1975  1661  156.00000   60  161  155.00000  158.00000  1.94%   916  724s
  1976  1661  158.00000   63  311  155.00000  158.00000  1.94%   916  725s
  1977  1662  156.00000  110  256  155.00000  158.00000  1.94%   915  730s
  1979  1663  156.00000   13  232  155.00000  158.00000  1.94%   914  736s
  1981  1665  156.00000  121  135  155.00000  158.00000  1.94%   914  741s
  1983  1666  156.50182  135  179  155.00000  158.00000  1.94%   913  748s
  1985  1667  157.00000   91  201  155.00000  158.00000  1.94%   912  754s
  1986  1668  156.00000  113  313  155.00000  158.00000  1.94%   911  755s
  1987  1669  156.00000  127  228  155.00000  158.00000  1.94%   911  760s
  1989  1670  157.00000  

 40208 25970  157.00000  119  448  155.00000  158.00000  1.94%   407 3256s
 40209 25971  157.86645   69  476  155.00000  158.00000  1.94%   407 3266s
 40210 25972  156.71506  526  552  155.00000  158.00000  1.94%   407 3273s
 40211 25972  158.00000  174  251  155.00000  158.00000  1.94%   407 3287s
 40212 25973  158.00000  605  461  155.00000  158.00000  1.94%   407 3293s
 40213 25974  156.00000  604  263  155.00000  158.00000  1.94%   407 3308s
 40214 25974  158.00000   58  423  155.00000  158.00000  1.94%   407 3313s
 40215 25975  156.44993  446  211  155.00000  158.00000  1.94%   407 3322s
 40216 25976  158.00000   41  432  155.00000  158.00000  1.94%   407 3326s
 40217 25976  157.76986  299  199  155.00000  158.00000  1.94%   407 3337s
 40218 25977  157.96207  119  452  155.00000  158.00000  1.94%   407 3342s
 40219 25978  158.00000   61  284  155.00000  158.00000  1.94%   407 3351s
 40220 25978  157.00000  498  612  155.00000  158.00000  1.94%   407 3356s
 40221 25979  156.23619  

## Entrada de datos: Instancia 4x200x2x3

In [16]:
datafile="4x200x2x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [17]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x1b4af5c4
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10492834 rows and 462380 columns (presolve time = 24s) ...
Presolve removed 10492834 rows and 462380 columns (presolve time = 29s) ...
Presolve removed 10531570 rows and 462444 columns (presolve time = 3

  2773  2553  162.00000   35  290  158.00000  162.00000  2.53%  1109  960s
  2899  2769  162.00000   45  290  158.00000  162.00000  2.53%  1094  973s
  3160  3028  162.00000   62  341  158.00000  162.00000  2.53%  1037  991s
  3507  3331  162.00000   90  254  158.00000  162.00000  2.53%   988 1012s
  3936  3224  162.00000  127  297  158.00000  162.00000  2.53%   931 1041s
  3972  3593  162.00000  130  267  158.00000  162.00000  2.53%   929 1065s
  4360  4527  162.00000  144  265  158.00000  162.00000  2.53%   896 1087s
  5476  5337  161.99364  196  282  158.00000  162.00000  2.53%   780 1105s
  6723  5428  161.92221  258  279  158.00000  162.00000  2.53%   683 1120s
  7265  5254  161.67197  365  266  158.00000  162.00000  2.53%   656 1140s
  7271  6320  161.34060  365  272  158.00000  162.00000  2.53%   656 1156s
  8341  7915  161.00000  450  240  158.00000  162.00000  2.53%   605 1173s
 10517  8950  159.54648  690  167  158.00000  162.00000  2.53%   506 1188s
 11662  9001  162.00000  

## Entrada de datos: Instancia 4x200x2x4

In [18]:
datafile="4x200x2x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [19]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x4390ccfe
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10447138 rows and 461472 columns (presolve time = 24s) ...
Presolve removed 10447138 rows and 461472 columns (presolve time = 29s) ...
Presolve removed 10485110 rows and 461476 columns (presolve time = 3

  3368  2650  157.47672   39   80  155.00000  158.14231  2.03%  58.9  371s
  3371  2652  157.00000   17  241  155.00000  158.14206  2.03%  58.9  375s
  3376  2655  156.50000   52  100  155.00000  158.14137  2.03%  58.8  381s
  3379  2657  156.50000   13  199  155.00000  158.14137  2.03%  58.7  385s
  3383  2660  156.00000  113   97  155.00000  158.14137  2.03%  58.6  390s
  3387  2664  156.00000  113  112  155.00000  158.14137  2.03%   103  395s
  3391  2667  156.00000   70  215  155.00000  158.14137  2.03%   103  400s
  3395  2670  157.50000   31  169  155.00000  158.13691  2.02%   103  405s
  3398  2672  156.00000  148  117  155.00000  158.13691  2.02%   103  410s
  3402  2674  156.00000  117  111  155.00000  158.13690  2.02%   103  415s
  3406  2677  157.00000   42   75  155.00000  158.13690  2.02%   103  420s
  3411  2681  156.00000  154   82  155.00000  158.13690  2.02%   124  426s
  3415  2684  157.50000   34  209  155.00000  158.13690  2.02%   124  430s
  3419  2687  156.00000  

## Entrada de datos: Instancia 4x200x2x5

In [20]:
datafile="4x200x2x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [21]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x5ac23218
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 16.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10532388 rows and 462844 columns (presolve time = 24s) ...
Presolve removed 10532388 rows and 462844 columns (presolve time = 29s) ...
Presolve removed 10570140 rows and 462912 columns (presolve time = 3

     0     0  162.55561    0 1338  154.00000  162.55561  5.56%     -  546s
     0     0  162.54883    0 1335  154.00000  162.54883  5.55%     -  550s
     0     0  162.54724    0 1394  154.00000  162.54724  5.55%     -  553s
     0     0  162.54655    0 1383  154.00000  162.54655  5.55%     -  554s
     0     0  162.54613    0 1402  154.00000  162.54613  5.55%     -  555s
     0     0  162.54581    0 1411  154.00000  162.54581  5.55%     -  557s
     0     0  162.54561    0 1427  154.00000  162.54561  5.55%     -  557s
     0     0  162.54554    0 1418  154.00000  162.54554  5.55%     -  558s
     0     0  162.51616    0 1280  154.00000  162.51616  5.53%     -  570s
H    0     0                     155.0000000  162.51616  4.85%     -  570s
     0     0  162.50913    0 1336  155.00000  162.50913  4.84%     -  575s
     0     0  162.50634    0 1408  155.00000  162.50634  4.84%     -  577s
     0     0  162.50558    0 1421  155.00000  162.50558  4.84%     -  579s
     0     0  162.50496  

     0     0  162.04254    0 1624  155.00000  162.04254  4.54%     -  856s
     0     0  162.04211    0 1629  155.00000  162.04211  4.54%     -  857s
     0     0  162.04199    0 1650  155.00000  162.04199  4.54%     -  857s
     0     0  162.03741    0 1596  155.00000  162.03741  4.54%     -  863s
     0     0  162.03460    0 1590  155.00000  162.03460  4.54%     -  866s
     0     0  162.03331    0 1579  155.00000  162.03331  4.54%     -  868s
     0     0  162.03298    0 1623  155.00000  162.03298  4.54%     -  869s
     0     0  162.02786    0 1566  155.00000  162.02786  4.53%     -  876s
     0     0  162.02622    0 1598  155.00000  162.02622  4.53%     -  879s
     0     0  162.02546    0 1630  155.00000  162.02546  4.53%     -  880s
     0     0  162.02513    0 1662  155.00000  162.02513  4.53%     -  881s
     0     0  162.02071    0 1594  155.00000  162.02071  4.53%     -  887s
     0     0  162.01959    0 1580  155.00000  162.01959  4.53%     -  889s
     0     0  162.01893  

  Relax-and-lift: 18

Explored 6450 nodes (4690170 simplex iterations) in 3601.14 seconds
Thread count was 6 (of 6 available processors)

Solution count 10: 157 156 155 ... 134

Time limit reached
Best objective 1.570000000000e+02, best bound 1.610000000000e+02, gap 2.5478%


## Entrada de datos: Instancia 4x200x2.5x1

In [22]:
datafile="4x200x2.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [23]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]
WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0xe17b6f21
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10541615 rows and 464600 columns (presolve time = 24s) ...
Presolve removed 10541615 rows and 464600 columns (presolve time = 29s) ...
Presolve removed 10584139 rows and 464656 columns (presolve time = 3

   611   722  146.15401   46  194  145.00000  147.28505  1.58%   930  626s
   855   857  146.15401   74  198  145.00000  147.28505  1.58%   856  675s
  1014  1001  146.00000   93  219  145.00000  147.28505  1.58%   821  705s
  1220  1096  146.94830   11  456  145.00000  147.28505  1.58%   817  733s
  1357  1097  146.84153   34  252  145.00000  147.28505  1.58%   812  754s
  1359  1098  146.28505   18  306  145.00000  147.28505  1.58%   810  758s
  1360  1099  146.26753   20  319  145.00000  147.28505  1.58%   810  761s
  1362  1100  146.00000  105  452  145.00000  147.28505  1.58%   809  768s
  1363  1101  146.00000   74  490  145.00000  147.28505  1.58%   808  772s
  1364  1102  147.12500    8  254  145.00000  147.28505  1.58%   807  778s
  1366  1103  147.15472   81  157  145.00000  147.28505  1.58%   806  782s
  1368  1104  146.00000   70  177  145.00000  147.00000  1.38%   805  786s
  1371  1106  147.00000  121  327  145.00000  147.00000  1.38%   803  790s
  1374  1108  146.00000  

 30649 20647  146.60856  101  453  145.00000  147.00000  1.38%   360 2226s
 31578 21384  146.54159  162  398  145.00000  147.00000  1.38%   361 2259s
 32698 21697  146.44807  233  402  145.00000  147.00000  1.38%   361 2303s
 33424 21501  146.21911  364  281  145.00000  147.00000  1.38%   361 2387s
 33472 21794  146.20516  369  299  145.00000  147.00000  1.38%   362 2486s
 33841 21989  146.17854  476  297  145.00000  147.00000  1.38%   360 2558s
 34193 23597  146.12984  554  294  145.00000  147.00000  1.38%   361 2590s
 36129 22952  146.80083  139  509  145.00000  147.00000  1.38%   352 2683s
 36141 23885  146.79512  140  510  145.00000  147.00000  1.38%   353 2714s
 37366 23829  146.57032  187  455  145.00000  147.00000  1.38%   352 2744s
 37745 24180  146.31529  312  352  145.00000  147.00000  1.38%   354 2777s
 38314 24406  146.83647  142  481  145.00000  147.00000  1.38%   358 2812s
 38788 24651  146.76167  154  387  145.00000  147.00000  1.38%   363 2846s
 39194 24517  146.84823  

## Entrada de datos: Instancia 4x200x2.5x2

In [24]:
datafile="4x200x2.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [25]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]
WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x0d0a55d2
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10607408 rows and 466284 columns (presolve time = 25s) ...
Presolve removed 10607408 rows and 466284 columns (presolve time = 30s) ...
Presolve removed 10651372 rows and 466288 columns (presolve time = 3

     0     0  146.51241    0  744  142.00000  146.51241  3.18%     -  372s
     0     0  146.46240    0  686  142.00000  146.46240  3.14%     -  379s
     0     0  146.45559    0  727  142.00000  146.45559  3.14%     -  382s
     0     0  146.45301    0  707  142.00000  146.45301  3.14%     -  383s
     0     0  146.45166    0  734  142.00000  146.45166  3.13%     -  384s
     0     0  146.45132    0  720  142.00000  146.45132  3.13%     -  384s
     0     0  146.45128    0  735  142.00000  146.45128  3.13%     -  384s
     0     0  146.37193    0  695  142.00000  146.37193  3.08%     -  392s
     0     0  146.36348    0  685  142.00000  146.36348  3.07%     -  395s
     0     0  146.35971    0  725  142.00000  146.35971  3.07%     -  398s
     0     0  146.35816    0  744  142.00000  146.35816  3.07%     -  399s
     0     0  146.35750    0  739  142.00000  146.35750  3.07%     -  400s
     0     0  146.35740    0  741  142.00000  146.35740  3.07%     -  400s
     0     0  146.31865  

     0     0  145.78702    0  668  142.00000  145.78702  2.67%     -  587s
     0     0  145.78629    0  694  142.00000  145.78629  2.67%     -  588s
     0     0  145.78574    0  681  142.00000  145.78574  2.67%     -  589s
     0     0  145.78561    0  679  142.00000  145.78561  2.67%     -  589s
     0     0  145.78174    0  679  142.00000  145.78174  2.66%     -  592s
     0     0  145.78124    0  674  142.00000  145.78124  2.66%     -  593s
     0     0  145.78046    0  676  142.00000  145.78046  2.66%     -  594s
     0     0  145.78030    0  687  142.00000  145.78030  2.66%     -  594s
     0     0  145.77676    0  669  142.00000  145.77676  2.66%     -  597s
     0     0  145.77563    0  681  142.00000  145.77563  2.66%     -  598s
     0     0  145.77541    0  710  142.00000  145.77541  2.66%     -  599s
     0     0  145.77228    0  686  142.00000  145.77228  2.66%     -  602s
     0     0  145.77181    0  687  142.00000  145.77181  2.66%     -  602s
     0     0  145.77169  

  3925  3788  144.99692  144  597  143.00000  145.33397  1.63%   599 1450s
  3931  3792  144.67327  474  604  143.00000  145.32836  1.63%   598 1455s
  3938  3796  144.68350  338  576  143.00000  145.32400  1.63%   597 1460s
  3943  3800  144.33846  536  578  143.00000  145.32028  1.62%   596 1465s
  3947  3802  144.51075  415  538  143.00000  145.27927  1.59%   596 1472s
  3952  3806  144.35265  362  641  143.00000  145.26719  1.59%   595 1476s
  3957  3809  145.25077   41  530  143.00000  145.25077  1.57%   594 1480s
  3963  3813  145.23624   56  568  143.00000  145.23624  1.56%   593 1486s
  3966  3815  144.78724  381  528  143.00000  145.22253  1.55%   593 1490s
  3971  3818  144.40684  437  622  143.00000  145.21174  1.55%   592 1495s
  3976  3822  144.54764  320  527  143.00000  145.21122  1.55%   591 1502s
  3978  3823  144.75316  278  552  143.00000  145.21118  1.55%   591 1505s
  3981  3826  145.04654  183  598  143.00000  145.21118  1.55%   688 1511s
  3984  3828  144.53461  

 12887  8845  144.14140  296  509  143.00000  145.17496  1.52%   685 2342s
 12891  8848  144.32472  195  596  143.00000  145.16922  1.52%   685 2345s
 12895  8851  144.17585  212  527  143.00000  145.16556  1.51%   685 2352s
 12900  8854  144.72961  213  622  143.00000  145.16390  1.51%   685 2355s
 12904  8857  145.07166   98  626  143.00000  145.15982  1.51%   684 2360s
 12909  8860  144.29628  412  648  143.00000  145.15820  1.51%   684 2365s
 12912  8862  144.12993  409  681  143.00000  145.15683  1.51%   684 2370s
 12915  8864  144.16916  407  663  143.00000  145.15624  1.51%   684 2375s
 12921  8868  144.23681  387  600  143.00000  145.14796  1.50%   683 2383s
 12923  8869  145.09778   52  661  143.00000  145.14595  1.50%   683 2385s
 12928  8873  144.96575  190  609  143.00000  145.14095  1.50%   683 2391s
 12932  8875  144.08798   82  636  143.00000  145.13810  1.50%   683 2395s
 12937  8879  144.45713  151  646  143.00000  145.13570  1.49%   683 2400s
 12940  8881  144.99093  

 22000 13750  144.36681  453  219  143.00000  145.05263  1.44%   607 3323s
 22225 13879  144.28089  510  151  143.00000  145.05263  1.44%   608 3353s
 22527 14004  144.30298  587  230  143.00000  145.05263  1.44%   608 3397s
 22764 14305  144.29567  633  212  143.00000  145.05263  1.44%   610 3427s
 23211 14324  144.17126  724  141  143.00000  145.04735  1.43%   609 3454s
 23408 14591  144.28009   66  533  143.00000  145.04735  1.43%   611 3485s
 23758 14779  144.05262  110  274  143.00000  145.04735  1.43%   613 3521s
 24130 14817  144.00534   66  600  143.00000  145.04734  1.43%   615 3554s
 24376 14907  144.31104   63  557  143.00000  145.04734  1.43%   617 3586s
 24572 14906  144.27780   74  507  143.00000  145.04734  1.43%   622 3600s

Cutting planes:
  Gomory: 13
  Cover: 86
  Projected implied bound: 1
  Clique: 100
  MIR: 786
  StrongCG: 305
  Flow cover: 348
  GUB cover: 11
  Zero half: 71
  RLT: 6
  Relax-and-lift: 12

Explored 24647 nodes (15675480 simplex iterations) in 360

## Entrada de datos: Instancia 4x200x2.5x3

In [26]:
datafile="4x200x2.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [27]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x465b5dcd
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 16.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 14s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 20s) ...
Presolve removed 10585742 rows and 465828 columns (presolve time = 26s) ...
Presolve removed 10585742 rows and 465828 columns (presolve time = 31s) ...
Presolve removed 10619766 rows and 465888 columns (presolve time = 3

     0     0  146.24023    0  951  141.00000  146.24023  3.72%     -  610s
     0     0  146.24008    0  956  141.00000  146.24008  3.72%     -  611s
     0     0  146.21480    0  850  141.00000  146.21480  3.70%     -  622s
     0     0  146.20833    0  935  141.00000  146.20833  3.69%     -  627s
     0     0  146.20629    0  950  141.00000  146.20629  3.69%     -  629s
     0     0  146.20607    0  947  141.00000  146.20607  3.69%     -  631s
     0     0  146.16537    0  853  141.00000  146.16537  3.66%     -  644s
     0     0  146.14803    0  927  141.00000  146.14803  3.65%     -  650s
     0     0  146.13508    0  843  141.00000  146.13508  3.64%     -  654s
     0     0  146.13429    0  914  141.00000  146.13429  3.64%     -  655s
     0     0  146.13406    0  895  141.00000  146.13406  3.64%     -  657s
     0     0  146.12209    0  874  141.00000  146.12209  3.63%     -  664s
     0     0  146.11793    0  897  141.00000  146.11793  3.63%     -  667s
     0     0  146.11690  

  1288  1162  145.74780   11  527  141.00000  145.75100  3.37%  1085 1457s
  1290  1163  143.67589   77  586  141.00000  145.75100  3.37%  1083 1462s
  1292  1164  144.44180   37  563  141.00000  145.75100  3.37%  1082 1467s
  1294  1166  145.75100    5  576  141.00000  145.75100  3.37%  1080 1472s
  1296  1167  145.64637   17  573  141.00000  145.75100  3.37%  1078 1477s
  1298  1168  142.73960   87  556  141.00000  145.75100  3.37%  1077 1481s
  1301  1170  144.19057   61  611  141.00000  145.75100  3.37%  1074 1486s
H 1301  1110                     142.0000000  145.75100  2.64%  1074 1491s
  1304  1112  145.74418    9  468  142.00000  145.75100  2.64%  1072 1495s
  1307  1114  143.78851   64  626  142.00000  145.75100  2.64%  1069 1501s
  1308  1115  144.19057   75  509  142.00000  145.75100  2.64%  1068 1510s
  1310  1116  144.41249   51  536  142.00000  145.75100  2.64%  1067 1515s
  1313  1118  144.37500   58  588  142.00000  145.75100  2.64%  1064 1521s
  1316  1120  145.47529  

 11940  9858  145.12686   87  514  142.00000  145.52089  2.48%  1108 3600s

Cutting planes:
  Gomory: 30
  Cover: 120
  Projected implied bound: 1
  Clique: 417
  MIR: 688
  StrongCG: 325
  Flow cover: 241
  GUB cover: 20
  Zero half: 149
  RLT: 1
  Relax-and-lift: 4

Explored 12101 nodes (14089664 simplex iterations) in 3601.22 seconds
Thread count was 6 (of 6 available processors)

Solution count 8: 142 141 138 ... -0

Time limit reached
Best objective 1.420000000000e+02, best bound 1.450000000000e+02, gap 2.1127%


## Entrada de datos: Instancia 4x200x2.5x4

In [28]:
datafile="4x200x2.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [29]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x2815e17d
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10585243 rows and 465720 columns (presolve time = 24s) ...
Presolve removed 10585243 rows and 465720 columns (presolve time = 29s) ...
Presolve removed 10626615 rows and 465724 columns (presolve time = 3

    68    75  147.71800   11  690  144.00000  148.16667  2.89%  2708  533s
    74    83  147.71800   12  709  144.00000  148.16667  2.89%  2612  536s
    82    96  147.71800   12  693  144.00000  148.16667  2.89%  2509  540s
   106   116  147.71800   15  648  144.00000  148.16667  2.89%  2239  551s
   115   130  147.71800   16  643  144.00000  148.16667  2.89%  2192  555s
   129   145  147.71800   17  645  144.00000  148.16667  2.89%  2095  560s
   144   151  147.71800   20  669  144.00000  148.16667  2.89%  2002  580s
   161   179  147.71800   21  679  144.00000  148.16667  2.89%  1936  589s
   178   192  147.71800   22  677  144.00000  148.16667  2.89%  1874  597s
   191   198  147.71800   23  634  144.00000  148.16667  2.89%  1902  626s
   197   215  147.71800   24  641  144.00000  148.16667  2.89%  1885  631s
   214   235  147.71800   28  631  144.00000  148.16667  2.89%  1845  636s
   234   253  147.04227   31  688  144.00000  148.16667  2.89%  1815  645s
   252   275  147.71800  

  1828  1661  146.41935   87  857  144.00000  147.65568  2.54%  1436 1553s
  1829  1662  147.05556   13  896  144.00000  147.64507  2.53%  1435 1556s
  1831  1663  146.63736   83  882  144.00000  147.62584  2.52%  1434 1563s
  1832  1664  147.38889   41  895  144.00000  147.61517  2.51%  1433 1566s
  1833  1665  146.62345   89  869  144.00000  147.60077  2.50%  1432 1572s
  1834  1665  146.92508   62  929  144.00000  147.59078  2.49%  1431 1578s
  1835  1666  146.09903   97  883  144.00000  147.53241  2.45%  1430 1585s
  1836  1667  147.05556   13  928  144.00000  147.49180  2.42%  1430 1591s
  1838  1668  146.50000   75  907  144.00000  147.46207  2.40%  1428 1599s
  1839  1669  145.21979   93  930  144.00000  147.45702  2.40%  1427 1602s
  1840  1669  147.35717   28  982  144.00000  147.43527  2.39%  1426 1606s
  1842  1671  147.42461   27  943  144.00000  147.42461  2.38%  1425 1612s
  1843  1671  147.30200   70  961  144.00000  147.41443  2.37%  1424 1617s
  1844  1672  147.38889  

  1972  1670  147.02943    9 1168  145.00000  147.02943  1.40%  1331 2125s
  1974  1672  147.02833   18 1137  145.00000  147.02833  1.40%  1330 2133s
  1975  1672  145.80062  122 1100  145.00000  147.02772  1.40%  1329 2140s
  1977  1674  147.02718   31 1112  145.00000  147.02718  1.40%  1328 2148s
  1978  1674  145.57569  119 1131  145.00000  147.02641  1.40%  1327 2153s
  1979  1675  147.02617    9 1117  145.00000  147.02617  1.40%  1326 2157s
  1980  1676  146.50000   64 1111  145.00000  147.02508  1.40%  1326 2162s
  1981  1676  147.02495   60 1128  145.00000  147.02495  1.40%  1325 2166s
  1982  1677  147.02403   34 1132  145.00000  147.02403  1.40%  1324 2170s
  1983  1678  147.02284   79 1081  145.00000  147.02284  1.40%  1324 2175s
  1985  1679  147.02129   25 1118  145.00000  147.02129  1.39%  1322 2185s
  1987  1680  147.02049   39 1121  145.00000  147.02049  1.39%  1321 2193s
  1988  1681  147.01984    4 1154  145.00000  147.01984  1.39%  1320 2197s
  1989  1682  145.73815  

  2116  1768  145.97551   61 1090  145.00000  146.90638  1.31%  1546 2698s
  2117  1769  146.06581  101 1102  145.00000  146.90587  1.31%  1546 2702s
  2118  1769  145.04186  108 1089  145.00000  146.90556  1.31%  1545 2706s
  2120  1771  146.90520   10 1078  145.00000  146.90520  1.31%  1543 2712s
  2121  1771  146.90492   65 1171  145.00000  146.90492  1.31%  1543 2715s
  2122  1772  146.32521   99 1089  145.00000  146.90436  1.31%  1542 2720s
  2124  1773  146.90278   78 1083  145.00000  146.90278  1.31%  1541 2727s
  2125  1774  146.90248   24 1101  145.00000  146.90248  1.31%  1540 2731s
  2126  1775  146.90185   14 1097  145.00000  146.90185  1.31%  1539 2735s
  2128  1776  146.41935   87 1090  145.00000  146.90119  1.31%  1538 2744s
  2129  1777  146.90089   13 1081  145.00000  146.90089  1.31%  1537 2747s
  2130  1777  146.90046   46 1081  145.00000  146.90046  1.31%  1536 2753s
  2131  1778  146.63736   83 1134  145.00000  146.90025  1.31%  1536 2757s
  2132  1779  146.89973  

  2262  1867  146.86902    6 1111  145.00000  146.86902  1.29%  1678 3283s
  2263  1867  146.86896   24 1100  145.00000  146.86896  1.29%  1677 3285s
  2265  1869  146.86818   27 1107  145.00000  146.86818  1.29%  1676 3292s
  2266  1869  145.42471  116 1065  145.00000  146.86789  1.29%  1675 3297s
  2267  1870  146.32521   99 1101  145.00000  146.86767  1.29%  1674 3300s
  2269  1871  146.86683   56 1058  145.00000  146.86683  1.29%  1673 3307s
  2270  1872  146.42998   49 1058  145.00000  146.86648  1.29%  1672 3311s
  2272  1873  146.86588    9 1074  145.00000  146.86588  1.29%  1671 3318s
  2273  1874  146.83333   22 1070  145.00000  146.86509  1.29%  1670 3321s
  2274  1875  146.86482   18 1036  145.00000  146.86482  1.29%  1669 3325s
  2276  1876  146.86409   54 1061  145.00000  146.86409  1.29%  1668 3330s
  2278  1877  145.57569  119 1037  145.00000  146.86371  1.29%  1666 3337s
  2279  1878  146.86359    9 1079  145.00000  146.86359  1.29%  1666 3340s
  2280  1879  146.50000  

## Entrada de datos: Instancia 4x200x2.5x5

In [30]:
datafile="4x200x2.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [31]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x16f0341c
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10607401 rows and 466272 columns (presolve time = 24s) ...
Presolve removed 10607401 rows and 466272 columns (presolve time = 29s) ...
Presolve removed 10644137 rows and 466324 columns (presolve time = 3

     0     0  147.47124    0  917  142.00000  147.47124  3.85%     -  341s
     0     0  147.45419    0  950  142.00000  147.45419  3.84%     -  345s
     0     0  147.44830    0  956  142.00000  147.44830  3.84%     -  347s
     0     0  147.44725    0  961  142.00000  147.44725  3.84%     -  348s
     0     0  147.44671    0  963  142.00000  147.44671  3.84%     -  349s
     0     0  147.44644    0  982  142.00000  147.44644  3.84%     -  349s
     0     0  147.38960    0  929  142.00000  147.38960  3.80%     -  357s
H    0     0                     143.0000000  147.38960  3.07%     -  360s
     0     0  147.38048    0  966  143.00000  147.38048  3.06%     -  360s
     0     0  147.37716    0  979  143.00000  147.37716  3.06%     -  361s
     0     0  147.37526    0  978  143.00000  147.37526  3.06%     -  362s
     0     0  147.37504    0  974  143.00000  147.37504  3.06%     -  363s
     0     0  147.32485    0  935  143.00000  147.32485  3.02%     -  369s
     0     0  147.31854  

   829   873  146.56787   87  673  143.00000  146.99798  2.80%   693  778s
   872   971  146.54964   94  644  143.00000  146.99798  2.80%   681  789s
   970  1044  146.51930  119  643  143.00000  146.99798  2.80%   648  800s
  1043  1113  146.50059  131  632  143.00000  146.99798  2.80%   631  812s
  1112  1119  146.48535  138  622  143.00000  146.99798  2.80%   617  827s
  1118  1237  146.47803  139  631  143.00000  146.99798  2.80%   615  839s
  1236  1356  146.43083  164  607  143.00000  146.99798  2.80%   587  853s
  1355  1542  146.39970  169  616  143.00000  146.99798  2.80%   562  865s
  1541  1607  146.37732  207  580  143.00000  146.99798  2.80%   518  878s
  1606  1733  146.36824  213  551  143.00000  146.99798  2.80%   517  890s
  1732  1893  146.33429  233  567  143.00000  146.99798  2.80%   505  903s
  1892  1923  146.29328  265  530  143.00000  146.99798  2.80%   487  920s
  1922  2051  146.28633  270  548  143.00000  146.99798  2.80%   487  932s
  2050  2176  146.23978  

  6542  6479  146.06924  168 1000  143.00000  146.06924  2.15%   275 1652s
  6544  6481  146.06732  170  991  143.00000  146.06732  2.14%   275 1656s
  6546  6482  144.89690  946  945  143.00000  146.06582  2.14%   275 1661s
  6548  6483  145.98452  337 1011  143.00000  146.06406  2.14%   275 1665s
  6551  6485  144.56271  801 1050  143.00000  146.06197  2.14%   275 1670s
  6554  6487  144.80902  673 1054  143.00000  146.05988  2.14%   275 1675s
  6557  6489  146.05648   88 1054  143.00000  146.05648  2.14%   275 1681s
  6559  6491  146.05494  311 1032  143.00000  146.05494  2.14%   275 1685s
  6562  6493  146.05342  202 1014  143.00000  146.05342  2.14%   275 1690s
  6566  6497  145.04894  558  848  143.00000  146.05236  2.13%   321 1695s
  6568  6498  144.70566  791  444  143.00000  146.05236  2.13%   321 1700s
  6570  6500  144.51775  937  662  143.00000  146.05236  2.13%   321 1707s
  6571  6500  146.02529  317  601  143.00000  146.05236  2.13%   321 1710s
  6573  6502  145.55747  

  6798  6653  144.30702 1064 1112  143.00000  145.83416  1.98%   342 2206s
  6800  6655  144.55166  875 1104  143.00000  145.83302  1.98%   342 2210s
  6803  6657  144.93366  607 1090  143.00000  145.83054  1.98%   342 2224s
  6804  6657  144.95612  593 1090  143.00000  145.82972  1.98%   342 2226s
  6806  6659  145.82830  194 1138  143.00000  145.82830  1.98%   342 2231s
  6808  6660  145.82525    8 1129  143.00000  145.82525  1.98%   342 2235s
  6810  6661  144.53792  892 1117  143.00000  145.82328  1.97%   342 2240s
  6813  6663  145.81993  237 1071  143.00000  145.81993  1.97%   342 2247s
  6815  6665  145.81709   33 1108  143.00000  145.81709  1.97%   342 2251s
  6818  6667  145.81585  299 1106  143.00000  145.81585  1.97%   341 2256s
  6820  6668  145.81384   58 1156  143.00000  145.81384  1.97%   341 2260s
  6822  6669  144.68561  806 1140  143.00000  145.81259  1.97%   341 2265s
  6825  6671  145.81191   55 1131  143.00000  145.81191  1.97%   341 2271s
  6827  6673  145.81147  

  7059  6830  145.71943  311  762  143.00000  145.71943  1.90%   398 2776s
  7061  6831  145.71943  171  781  143.00000  145.71943  1.90%   398 2781s
  7063  6833  145.71943  159  757  143.00000  145.71943  1.90%   398 2786s
  7065  6834  144.92729  895  754  143.00000  145.71943  1.90%   397 2790s
  7069  6837  145.71943  371  834  143.00000  145.71943  1.90%   397 2796s
  7071  6838  145.71943  317  833  143.00000  145.71943  1.90%   397 2801s
  7073  6839  145.55747  531  855  143.00000  145.71943  1.90%   397 2805s
  7075  6841  145.39869  758  885  143.00000  145.71943  1.90%   397 2810s
  7077  6842  144.43453 1039  864  143.00000  145.71943  1.90%   397 2817s
  7078  6843  145.63681  672  869  143.00000  145.71943  1.90%   397 2820s
  7081  6845  145.60052  561  875  143.00000  145.71943  1.90%   397 2825s
  7083  6846  145.71943  385  915  143.00000  145.71943  1.90%   396 2843s
  7084  6847  145.71943  297  941  143.00000  145.71943  1.90%   396 2845s
  7087  6849  145.41240  

## Entrada de datos: Instancia 4x200x3x1

In [32]:
datafile="4x200x3x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [33]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x1bb8999d
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 14s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 19s) ...
Presolve removed 10677339 rows and 468892 columns (presolve time = 25s) ...
Presolve removed 10677339 rows and 468892 columns (presolve time = 29s) ...
Presolve removed 10721219 rows and 468892 columns (presolve time = 3

     0     0  127.03373    0  493  123.00000  127.03373  3.28%     -  276s
     0     0  127.03150    0  481  123.00000  127.03150  3.28%     -  278s
     0     0  127.02697    0  486  123.00000  127.02697  3.27%     -  280s
     0     0  127.02623    0  516  123.00000  127.02623  3.27%     -  281s
     0     0  127.02552    0  520  123.00000  127.02552  3.27%     -  282s
     0     0  127.02506    0  509  123.00000  127.02506  3.27%     -  284s
     0     0  127.02498    0  510  123.00000  127.02498  3.27%     -  284s
     0     0  127.02383    0  519  123.00000  127.02383  3.27%     -  286s
     0     0  127.01948    0  508  123.00000  127.01948  3.27%     -  288s
     0     0  127.01635    0  478  123.00000  127.01635  3.27%     -  290s
     0     0  127.01527    0  485  123.00000  127.01527  3.26%     -  291s
     0     0  127.01484    0  503  123.00000  127.01484  3.26%     -  291s
     0     0  127.01476    0  494  123.00000  127.01476  3.26%     -  292s
     0     0  127.01312  

     0     0  126.81682    0  538  124.00000  126.81682  2.27%     -  516s
     0     0  126.81670    0  525  124.00000  126.81670  2.27%     -  518s
     0     0  126.81594    0  512  124.00000  126.81594  2.27%     -  520s
     0     0  126.79536    0  514  124.00000  126.79536  2.25%     -  524s
     0     0  126.79457    0  497  124.00000  126.79457  2.25%     -  526s
     0     0  126.79449    0  452  124.00000  126.79449  2.25%     -  526s
     0     0  126.79448    0  445  124.00000  126.79448  2.25%     -  528s
     0     0  126.79439    0  462  124.00000  126.79439  2.25%     -  529s
     0     0  126.79436    0  452  124.00000  126.79436  2.25%     -  530s
     0     0  126.79436    0  323  124.00000  126.79436  2.25%     -  531s
     0     2  126.79436    0  279  124.00000  126.79436  2.25%     -  539s
     3     8  126.79272    2  386  124.00000  126.79436  2.25%   436  540s
    19    26  126.79269    4  458  124.00000  126.79433  2.25%  1181  545s
    37    45  126.79269  

  5798  3990  125.85655  538  260  124.00000  126.71249  2.19%  1085 1354s
  5942  4100  125.79954  573  261  124.00000  126.71249  2.19%  1096 1375s
  6164  4127  125.00000  657  133  124.00000  126.68029  2.16%  1100 1394s
  6241  4186  126.60222   23  620  124.00000  126.68029  2.16%  1129 1416s
  6300  4264  126.54494   43  554  124.00000  126.68029  2.16%  1146 1436s
  6379  4319  126.54376   44  525  124.00000  126.68029  2.16%  1158 1459s
  6447  4388  126.48116   60  494  124.00000  126.68029  2.16%  1189 1483s
  6516  4451  126.42260   79  465  124.00000  126.68029  2.16%  1207 1506s
  6579  4547  126.33881   92  468  124.00000  126.68029  2.16%  1226 1534s
  6686  4594  126.19385  115  450  124.00000  126.68029  2.16%  1250 1563s
  6733  4665  126.06221  132  401  124.00000  126.68029  2.16%  1267 1589s
  6804  4758  125.69902  154  342  124.00000  126.68029  2.16%  1291 1609s
  6901  4850  125.02014  198  255  124.00000  126.66003  2.15%  1310 1635s
  7003  4990  126.53785  

 12970  9888  126.15035   36  476  124.00000  126.15800  1.74%  1622 3198s
 12982  9901  126.14992   37  492  124.00000  126.15800  1.74%  1622 3208s
 12999  9914  126.11979   38  475  124.00000  126.15800  1.74%  1625 3216s
 13017  9924  126.14557   40  477  124.00000  126.15800  1.74%  1625 3224s
 13033  9942  126.10056   41  451  124.00000  126.15800  1.74%  1626 3232s
 13057  9959  126.12972   43  481  124.00000  126.15800  1.74%  1626 3242s
 13082  9979  126.13585   45  448  124.00000  126.15800  1.74%  1626 3251s
 13110 10001  126.13255   48  479  124.00000  126.15800  1.74%  1627 3261s
 13141 10022  126.13027   51  456  124.00000  126.15800  1.74%  1628 3270s
 13173 10028  126.11743   54  479  124.00000  126.15800  1.74%  1628 3278s
 13189 10063  126.11891   54  460  124.00000  126.15800  1.74%  1628 3288s
 13230 10095  126.11641   57  423  124.00000  126.15800  1.74%  1628 3298s
 13275 10126  126.11285   59  480  124.00000  126.15800  1.74%  1627 3308s
 13321 10154  126.10459  

## Entrada de datos: Instancia 4x200x3x2

In [34]:
datafile="4x200x3x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [35]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x418a52af
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10644218 rows and 467908 columns (presolve time = 24s) ...
Presolve removed 10644218 rows and 467908 columns (presolve time = 29s) ...
Presolve removed 10686486 rows and 467944 columns (presolve time = 3

  2034  1543  123.00000   99  131  122.00000  123.00000  0.82%   262  346s
  2037  1550  123.00000   14  154  122.00000  123.00000  0.82%   316  351s
  2047  1559  123.00000   16  158  122.00000  123.00000  0.82%   315  356s
  2065  1571  123.00000   17  157  122.00000  123.00000  0.82%   316  360s
  2091  1608  123.00000   19  160  122.00000  123.00000  0.82%   315  365s
  2117  1653  123.00000   21  103  122.00000  123.00000  0.82%   319  370s
  2170  1832  123.00000   26  113  122.00000  123.00000  0.82%   323  385s
  2371  2956  123.00000   36  131  122.00000  123.00000  0.82%   318  402s
  3883  2596  123.00000   95  304  122.00000  123.00000  0.82%   282  410s
  4057  2830  123.00000  116  259  122.00000  123.00000  0.82%   286  416s
  4424  2807  123.00000  160  289  122.00000  123.00000  0.82%   288  424s
  4593  2821 infeasible  172       122.00000  123.00000  0.82%   304  434s
  4696  3078  123.00000   42  259  122.00000  123.00000  0.82%   308  442s
  5053  3032  123.00000  

 38227 15655  123.00000   41  261  122.00000  123.00000  0.82%   668 2460s
 38233 15657 infeasible   42       122.00000  123.00000  0.82%   668 2473s
 38239 15661  123.00000   42  281  122.00000  123.00000  0.82%   668 2479s
 38245 15665  123.00000   43  255  122.00000  123.00000  0.82%   668 2483s
 38251 15667  123.00000   43  290  122.00000  123.00000  0.82%   668 2493s
 38259 15671  123.00000   44  307  122.00000  123.00000  0.82%   668 2497s
 38265 15674  123.00000   44  276  122.00000  123.00000  0.82%   669 2501s
 38271 15677  123.00000   45  327  122.00000  123.00000  0.82%   669 2505s
 38277 15682  123.00000   45  311  122.00000  123.00000  0.82%   669 2510s
 38291 15720  123.00000   46  326  122.00000  123.00000  0.82%   669 2527s
 38327 15803  123.00000   49  179  122.00000  123.00000  0.82%   669 2557s
 38425 16285  123.00000   57  344  122.00000  123.00000  0.82%   669 2618s
 39018 16737  123.00000   45  374  122.00000  123.00000  0.82%   667 2668s
 39770 17132     cutoff  

## Entrada de datos: Instancia 4x200x3x3

In [36]:
datafile="4x200x3x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [37]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Academic license - for non-commercial use only - expires 2021-06-11
Using license file C:\Users\elPatan03\gurobi.lic
Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0xfdc41f39
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 17.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10680030 rows and 468876 columns (presolve time = 25s) ...
Presolve removed 10680030 r

     0     0  126.29010    0  690  124.00000  126.29010  1.85%     -  325s
     0     0  126.28994    0  688  124.00000  126.28994  1.85%     -  326s
     0     0  126.28991    0  688  124.00000  126.28991  1.85%     -  328s
     0     0  126.28867    0  682  124.00000  126.28867  1.85%     -  331s
     0     0  126.28772    0  656  124.00000  126.28772  1.84%     -  334s
     0     0  126.28637    0  691  124.00000  126.28637  1.84%     -  337s
     0     0  126.28589    0  677  124.00000  126.28589  1.84%     -  339s
     0     0  126.28525    0  712  124.00000  126.28525  1.84%     -  341s
     0     0  126.28514    0  666  124.00000  126.28514  1.84%     -  342s
     0     0  126.28162    0  659  124.00000  126.28162  1.84%     -  345s
     0     0  126.27986    0  711  124.00000  126.27986  1.84%     -  347s
     0     0  126.27947    0  703  124.00000  126.27947  1.84%     -  349s
     0     0  126.27928    0  737  124.00000  126.27928  1.84%     -  350s
     0     0  126.27910  

    31    32  126.17857    6  779  124.00000  126.17910  1.76%  5425  636s
    37    40  126.17871    6  803  124.00000  126.17910  1.76%  4869  641s
    45    49  126.14118    7  712  124.00000  126.17910  1.76%  4622  648s
    54    58  126.14096    8  690  124.00000  126.17910  1.76%  4198  652s
    63    67  126.14096    9  688  124.00000  126.17910  1.76%  3825  657s
    74    96  126.13775   10  663  124.00000  126.17910  1.76%  3583  669s
   103   114  126.12559   13  620  124.00000  126.17910  1.76%  3050  678s
   121   158  126.12559   13  616  124.00000  126.17910  1.76%  2853  691s
   167   180  126.09640   19  565  124.00000  126.17910  1.76%  2477  698s
   189   208  126.09621   20  572  124.00000  126.17910  1.76%  2347  708s
   219   257  126.07811   22  566  124.00000  126.17910  1.76%  2239  719s
   270   318  126.09474   26  559  124.00000  126.17910  1.76%  2030  730s
   331   385  126.09007   32  553  124.00000  126.17910  1.76%  1853  738s
   408   388  126.01181  

  9855  7425  126.00000  132  315  125.00000  126.13078  0.90%  1708 2564s
 10270  7864  126.00786  112  379  125.00000  126.13063  0.90%  1728 2642s
 10781  8463  126.08852   72  539  125.00000  126.13063  0.90%  1743 2718s
 11516  8870  126.01683  223  342  125.00000  126.13053  0.90%  1720 2803s
 12072  9055     cutoff   41       125.00000  126.13007  0.90%  1733 2903s
 12344  9313  126.09547   55  611  125.00000  126.12988  0.90%  1778 3003s
 12603  9314  126.07126  174  707  125.00000  126.12988  0.90%  1821 3029s
 12605  9315  126.08532   68  236  125.00000  126.12988  0.90%  1821 3031s
 12609  9318  126.05177  146  560  125.00000  126.12988  0.90%  1820 3035s
 12613  9321  126.03707  216  580  125.00000  126.12988  0.90%  1820 3041s
 12615  9322  126.05770   57  615  125.00000  126.12988  0.90%  1819 3045s
 12617  9323  126.09253   48  642  125.00000  126.12988  0.90%  1819 3052s
 12619  9325  126.01377  315  667  125.00000  126.12988  0.90%  1819 3055s
 12621  9326  126.01811  

## Entrada de datos: Instancia 4x200x3x4

In [38]:
datafile="4x200x3x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [39]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0xa93ee62d
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10657719 rows and 468308 columns (presolve time = 24s) ...
Presolve removed 10657719 rows and 468308 columns (presolve time = 29s) ...
Presolve removed 10701319 rows and 468348 columns (presolve time = 3

  2106  1704  127.00000   19  169  126.00000  127.45455  1.15%   467  395s
  2216  1770  127.00000   26  253  126.00000  127.45455  1.15%   456  400s
  2449  1884     cutoff   42       126.00000  127.45455  1.15%   433  405s
  2705  2020  127.35000   60  236  126.00000  127.45455  1.15%   417  410s
  2982  2150  127.00000   84  180  126.00000  127.45455  1.15%   405  425s
  3167  2297  127.00000  100  205  126.00000  127.45455  1.15%   401  430s
  3431  2495  127.00000  118  218  126.00000  127.45455  1.15%   397  435s
  3858  2758  127.00000  145  228  126.00000  127.45455  1.15%   384  441s
  4342  3032     cutoff  166       126.00000  127.45455  1.15%   364  445s
  4893  3200  127.25000   69  174  126.00000  127.45455  1.15%   341  457s
  5036  3257  127.25000  101  199  126.00000  127.45455  1.15%   337  460s
  5565  3561  127.00000   65  236  126.00000  127.45455  1.15%   325  466s
  5884  3749  127.00000  143  238  126.00000  127.45455  1.15%   323  471s
  6042  3756  127.00000  

 24235 15475  127.00000  168  203  126.00000  127.40000  1.11%   441 1274s
 24589 15637  127.00000  250  355  126.00000  127.40000  1.11%   445 1295s
 24929 15818  127.35000   53  445  126.00000  127.40000  1.11%   449 1318s
 25356 15964     cutoff  101       126.00000  127.40000  1.11%   453 1351s
 25706 17050  127.00000  160  401  126.00000  127.40000  1.11%   456 1390s
 26981 17069  127.20000  150  351  126.00000  127.40000  1.11%   456 1412s
 27480 17165  127.00000  241  371  126.00000  127.40000  1.11%   458 1449s
 27816 17422  127.00000   84  491  126.00000  127.40000  1.11%   463 1478s
 28217 17619     cutoff  166       126.00000  127.40000  1.11%   468 1511s
 28616 17978  127.00000   97  496  126.00000  127.40000  1.11%   475 1547s
 29146 18275  127.00000   53  543  126.00000  127.40000  1.11%   480 1583s
 29751 18528     cutoff   60       126.00000  127.40000  1.11%   485 1618s
 30246 18827  127.00000   94  533  126.00000  127.40000  1.11%   488 1648s
 30773 19157  127.00000  

 51574 28687  127.40000   64  305  126.00000  127.40000  1.11%   511 3071s
 51599 28694  127.40000   66  266  126.00000  127.40000  1.11%   511 3077s
 51608 28701  127.40000   66  287  126.00000  127.40000  1.11%   511 3080s
 51661 28714  127.40000   69  317  126.00000  127.40000  1.11%   511 3087s
 51684 28719  127.40000   70  305  126.00000  127.40000  1.11%   511 3090s
 51766 28747  127.00000   72  321  126.00000  127.40000  1.11%   510 3097s
 51795 28752  127.40000   73  342  126.00000  127.40000  1.11%   510 3100s
 51843 28770  127.38462   76  324  126.00000  127.40000  1.11%   510 3107s
 51865 28778  127.35000   77  394  126.00000  127.40000  1.11%   510 3110s
 51933 28804  127.35000   79  287  126.00000  127.40000  1.11%   510 3117s
 51973 28826  127.35000   80  320  126.00000  127.40000  1.11%   510 3121s
 52080 28874  127.35000   82  387  126.00000  127.40000  1.11%   509 3127s
 52135 28896  127.35000   83  462  126.00000  127.40000  1.11%   509 3131s
 52251 28960  127.00000  

## Entrada de datos: Instancia 4x200x3x5

In [40]:
datafile="4x200x3x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [41]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0xaede31b2
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 17.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10653136 rows and 468068 columns (presolve time = 24s) ...
Presolve removed 10653136 rows and 468068 columns (presolve time = 29s) ...
Presolve removed 10690284 rows and 468108 columns (presolve time = 3

     0     0  128.50000    0  453  127.00000  128.50000  1.18%     -  351s
     0     0  128.50000    0  502  127.00000  128.50000  1.18%     -  354s
     0     0  128.50000    0  573  127.00000  128.50000  1.18%     -  359s
     0     0  128.50000    0  572  127.00000  128.50000  1.18%     -  364s
     0     0  128.50000    0  549  127.00000  128.50000  1.18%     -  368s
     0     0  128.50000    0  566  127.00000  128.50000  1.18%     -  370s
     0     0  128.50000    0  603  127.00000  128.50000  1.18%     -  372s
     0     0  128.50000    0  544  127.00000  128.50000  1.18%     -  375s
     0     0  128.50000    0  614  127.00000  128.50000  1.18%     -  378s
     0     0  128.50000    0  636  127.00000  128.50000  1.18%     -  379s
     0     0  128.49793    0  613  127.00000  128.49793  1.18%     -  382s
     0     0  128.49515    0  619  127.00000  128.49515  1.18%     -  384s
     0     0  128.48773    0  622  127.00000  128.48773  1.17%     -  386s
     0     0  128.48534  

     0     0  128.29953    0  585  127.00000  128.29953  1.02%     -  525s
     0     0  128.29896    0  600  127.00000  128.29896  1.02%     -  526s
     0     0  128.29884    0  566  127.00000  128.29884  1.02%     -  526s
     0     0  128.29711    0  598  127.00000  128.29711  1.02%     -  528s
     0     0  128.29662    0  598  127.00000  128.29662  1.02%     -  529s
     0     0  128.29660    0  587  127.00000  128.29660  1.02%     -  529s
     0     0  128.29606    0  574  127.00000  128.29606  1.02%     -  530s
     0     0  128.29606    0  489  127.00000  128.29606  1.02%     -  530s
     0     2  128.29602    0  489  127.00000  128.29602  1.02%     -  533s
     7    14  128.29413    3  597  127.00000  128.29420  1.02%  1297  535s
    31    38  128.29277    6  555  127.00000  128.29382  1.02%  1903  540s
    75   104  128.29274   11  564  127.00000  128.29382  1.02%  1347  547s
   103   202  128.29219   13  562  127.00000  128.29382  1.02%  1199  554s
   201   362  128.29116  

 25747 15351  128.00000   50  303  127.00000  128.00000  0.79%   938 2934s
 26136 15493  128.00000   80  189  127.00000  128.00000  0.79%   931 2999s
 26431 16062  128.00000   76  217  127.00000  128.00000  0.79%   927 3073s
 27207 16378  128.00000  196  228  127.00000  128.00000  0.79%   915 3113s
 27866 16467  128.00000  346  201  127.00000  128.00000  0.79%   909 3164s
 28220 17273  128.00000   59  208  127.00000  128.00000  0.79%   907 3207s
 29266 17940  128.00000   90  246  127.00000  128.00000  0.79%   900 3245s
 30432 18473  128.00000   68  164  127.00000  128.00000  0.79%   890 3278s
 31466 18818  128.00000  133  249  127.00000  128.00000  0.79%   882 3309s
 32243 19505  128.00000   95  178  127.00000  128.00000  0.79%   881 3341s
 33373 19818     cutoff  206       127.00000  128.00000  0.79%   873 3372s
 34158 19824     cutoff  156       127.00000  128.00000  0.79%   867 3440s
 34516 19677     cutoff  112       127.00000  128.00000  0.79%   872 3544s
 34556 19846  128.00000  

## Entrada de datos: Instancia 4x200x3.5x1

In [42]:
datafile="4x200x3.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [43]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Academic license - for non-commercial use only - expires 2021-06-11
Using license file C:\Users\elPatan03\gurobi.lic
Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0xa822a829
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 14s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 19s) ...
Presolve removed 10692500 rows and 469408 columns (presolve time = 25s) ...
Presolve removed 10730588 r

   441   724  124.50000   45  425  122.00000  125.00000  2.46%  1305  366s
H  494   670                     123.0000000  125.00000  1.63%  1232  366s
   806   677  125.00000    9  547  123.00000  125.00000  1.63%  1001  393s
  1012   971  125.00000   27  531  123.00000  125.00000  1.63%   989  429s
  1395  1386  124.91265   89  499  123.00000  125.00000  1.63%   899  470s
  1914  1387  124.07020  214  253  123.00000  125.00000  1.63%   789  483s
  1917  1389  124.49159   42  259  123.00000  125.00000  1.63%   788  485s
  1921  1392  124.50000   44  473  123.00000  125.00000  1.63%   786  491s
  1923  1393  124.00000   99  345  123.00000  125.00000  1.63%   785  496s
  1925  1394  125.00000    8  314  123.00000  125.00000  1.63%   784  502s
  1927  1396  125.00000   15  322  123.00000  125.00000  1.63%   784  506s
  1929  1397  124.88838   42  328  123.00000  125.00000  1.63%   783  512s
  1931  1398  124.27532   48  423  123.00000  124.98273  1.61%   782  516s
  1934  1400  124.94984  

  2437  1745  124.11820   55  208  123.00000  124.11820  0.91%  1203 1005s
  2447  1753  124.11820   71  421  123.00000  124.11820  0.91%  1223 1010s
  2452  1757  124.11820   44  297  123.00000  124.11820  0.91%  1220 1015s
  2461  1763  124.11692   47  230  123.00000  124.11692  0.91%  1216 1020s
  2470  1769  124.00000   16  313  123.00000  124.11692  0.91%  1211 1025s
  2479  1776  124.11692   45  370  123.00000  124.11692  0.91%  1237 1030s
  2484  1780  124.11684   34  310  123.00000  124.11684  0.91%  1235 1035s
  2491  1784  124.10864   43  198  123.00000  124.10864  0.90%  1231 1040s
  2501  1791  124.10829  202  179  123.00000  124.10829  0.90%  1226 1045s
  2512  1800  124.10829   96  390  123.00000  124.10829  0.90%  1251 1050s
  2519  1804  124.10829   14  119  123.00000  124.10829  0.90%  1248 1055s
  2530  1812  124.10825   20  201  123.00000  124.10825  0.90%  1242 1060s
  2537  1815  124.10763  132  190  123.00000  124.10763  0.90%   664 1067s
  2547  1818  124.10733  

 23583 11812  124.03610  194  394  123.00000  124.08052  0.88%   483 1707s
 23814 12152  124.00550  221  346  123.00000  124.08050  0.88%   487 1730s
 24346 12353  124.04412  188  367  123.00000  124.08050  0.88%   490 1752s
 24835 12453  124.01215  229  463  123.00000  124.08050  0.88%   494 1776s
 25200 12765  123.99999  239  449  123.00000  124.07855  0.88%   505 1807s
 25769 13087  124.02388  162  449  123.00000  124.07748  0.88%   516 1850s
 26495 13295     cutoff  167       123.00000  124.07674  0.88%   534 1900s
 27099 13690  124.03680  186  411  123.00000  124.07614  0.87%   552 1954s
 27874 14918  124.02295  188  408  123.00000  124.07551  0.87%   564 2009s
 29791 14708     cutoff  171       123.00000  124.07495  0.87%   574 2047s
 30351 15219  124.02619  175  351  123.00000  124.07447  0.87%   589 2094s
 30830 15059  124.05261  176  299  123.00000  124.07409  0.87%   594 2095s
 31326 15304  124.03829  212  278  123.00000  124.07407  0.87%   598 2140s
 31940 15541  124.05192  

## Entrada de datos: Instancia 4x200x3.5x2

In [44]:
datafile="4x200x3.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [45]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x03ceefff
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10696187 rows and 469616 columns (presolve time = 24s) ...
Presolve removed 10696187 rows and 469616 columns (presolve time = 29s) ...
Presolve removed 10735867 rows and 469656 columns (presolve time = 3

  1418  1179  116.22304   39  201  115.00000  116.65138  1.44%   483  335s
  1423  1182  116.03808   56  302  115.00000  116.65131  1.44%   481  340s
  1428  1186  116.45306   47  157  115.00000  116.65099  1.44%   480  345s
  1435  1192  116.65099   22  231  115.00000  116.65099  1.44%   534  357s
  1439  1195  116.65099   55  306  115.00000  116.65099  1.44%   532  360s
  1444  1198  116.63904   19  129  115.00000  116.64988  1.43%   531  365s
  1449  1201  116.64988   19  248  115.00000  116.64988  1.43%   529  370s
  1454  1205  116.33442   49  145  115.00000  116.64988  1.43%   527  376s
  1459  1210  116.64988   24  151  115.00000  116.64988  1.43%   556  381s
  1468  1214  116.64988   27  178  115.00000  116.64988  1.43%   554  387s
  1486  1221  116.04506   29  251  115.00000  116.64988  1.43%   549  390s
  1522  1232  116.64988   32  173  115.00000  116.64988  1.43%   538  404s
  1558  1244  116.64988   34  178  115.00000  116.64988  1.43%   528  405s
  1599  1260  116.34932  

## Entrada de datos: Instancia 4x200x3.5x3

In [46]:
datafile="4x200x3.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [47]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x5936dc5e
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10671164 rows and 469048 columns (presolve time = 24s) ...
Presolve removed 10671164 rows and 469048 columns (presolve time = 29s) ...
Presolve removed 10707652 rows and 469084 columns (presolve time = 3

     0     0  120.06552    0  447  118.00000  120.06552  1.75%     -  256s
     0     0  120.06533    0  454  118.00000  120.06533  1.75%     -  256s
     0     0  120.01810    0  443  118.00000  120.01810  1.71%     -  258s
     0     0  120.01548    0  402  118.00000  120.01548  1.71%     -  259s
     0     0  120.01362    0  435  118.00000  120.01362  1.71%     -  260s
     0     0  120.01362    0  402  118.00000  120.01362  1.71%     -  260s
     0     0  119.98455    0  426  118.00000  119.98455  1.68%     -  261s
     0     0  119.98321    0  433  118.00000  119.98321  1.68%     -  262s
     0     0  119.98305    0  422  118.00000  119.98305  1.68%     -  262s
     0     0  119.97633    0  458  118.00000  119.97633  1.67%     -  263s
     0     0  119.97220    0  442  118.00000  119.97220  1.67%     -  264s
     0     0  119.97073    0  423  118.00000  119.97073  1.67%     -  265s
     0     0  119.97055    0  435  118.00000  119.97055  1.67%     -  265s
     0     0  119.96508  

## Entrada de datos: Instancia 4x200x3.5x4

In [48]:
datafile="4x200x3.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [49]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x9f3c63c0
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10673273 rows and 469160 columns (presolve time = 24s) ...
Presolve removed 10673273 rows and 469160 columns (presolve time = 29s) ...
Presolve removed 10710125 rows and 469160 columns (presolve time = 3

  2026  2102  119.00000   31  118  117.00000  119.00000  1.71%   420  429s
  2593  2771  118.99124  100  135  117.00000  119.00000  1.71%   388  439s
  3519  3272     cutoff  229       117.00000  119.00000  1.71%   333  451s
  4542  3452  119.00000   79  187  117.00000  119.00000  1.71%   278  456s
  5182  3492  119.00000  101  240  117.00000  119.00000  1.71%   259  478s
  5534  3686  119.00000  102  242  117.00000  119.00000  1.71%   247  483s
  6014  3725  119.00000  120  230  117.00000  119.00000  1.71%   241  506s
  6181  4128  119.00000  121  202  117.00000  119.00000  1.71%   236  512s
  6615  4540  119.00000  164  235  117.00000  119.00000  1.71%   238  538s
  7067  5086  119.00000  200  175  117.00000  119.00000  1.71%   240  544s
  7622  5876  119.00000  236  271  117.00000  119.00000  1.71%   242  552s
  8455  6323  118.39129  467  242  117.00000  119.00000  1.71%   235  559s
  9046  6781  119.00000   59  177  117.00000  119.00000  1.71%   237  567s
  9591  7155  119.00000  

## Entrada de datos: Instancia 4x200x3.5x5

In [50]:
datafile="4x200x3.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [51]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x8a42ec5e
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 17.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 14s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 19s) ...
Presolve removed 10699555 rows and 469812 columns (presolve time = 25s) ...
Presolve removed 10699555 rows and 469812 columns (presolve time = 29s) ...
Presolve removed 10747595 rows and 469812 columns (presolve time = 3

  8580  6225  120.55153   51  372  119.00000  120.64440  1.38%   173  346s
  8884  6594  120.52900   62  380  119.00000  120.64440  1.38%   178  351s
  9417  6874  120.52505   80  327  119.00000  120.64440  1.38%   183  358s
  9462  6947  120.52336   84  350  119.00000  120.64440  1.38%   186  361s
  9742  7405  120.47128   98  398  119.00000  120.64440  1.38%   191  368s
 10068  7670  120.49962  106  351  119.00000  120.64440  1.38%   192  371s
 10342  7854  120.49365  117  347  119.00000  120.64440  1.38%   193  375s
 10749  8189  120.44266  140  357  119.00000  120.64440  1.38%   200  382s
 10920  8357  120.42222  149  348  119.00000  120.64440  1.38%   202  386s
 11411  8926  120.39088  171  330  119.00000  120.64440  1.38%   207  393s
 11674  9257  120.19265  184  340  119.00000  120.64440  1.38%   208  397s
 12023  9518  120.18950  203  312  119.00000  120.64440  1.38%   210  401s
 12306  9923  120.18407  223  301  119.00000  120.64440  1.38%   212  406s
 12745  9924  120.25806  

 37978 23750  120.29559  323  367  119.00000  120.57707  1.33%   564 1903s
 37979 23751  120.21494  229  475  119.00000  120.57707  1.33%   564 1906s
 37980 23752  120.32226  333  355  119.00000  120.57707  1.33%   564 1910s
 37982 23753  120.45550  120  429  119.00000  120.57707  1.33%   564 1915s
 37984 23754  120.37391  237  427  119.00000  120.57707  1.33%   564 1920s
 37986 23756  120.15617  443  419  119.00000  120.57707  1.33%   564 1926s
 37988 23757  120.19597  176  430  119.00000  120.57707  1.33%   564 1931s
 37990 23758  120.35595   97  434  119.00000  120.57707  1.33%   564 1937s
 37992 23760  120.53241   87  449  119.00000  120.57707  1.33%   564 1942s
 37993 23760  120.00000  204  537  119.00000  120.57707  1.33%   564 1945s
 37995 23762  120.09035  522  533  119.00000  120.57707  1.33%   563 1951s
 37997 23763  120.47826   34  500  119.00000  120.57707  1.33%   563 1956s
 37998 23764  120.21880  364  482  119.00000  120.57707  1.33%   563 1960s
 38000 23765  120.45086  

 38391 24023  120.00000   66  211  119.00000  120.41829  1.19%   568 2458s
 38402 24040  120.21545   68  253  119.00000  120.41829  1.19%   568 2460s
 38439 24064  120.21094   72  249  119.00000  120.41829  1.19%   568 2465s
 38492 24093  120.18750   80  244  119.00000  120.41829  1.19%   568 2470s
 38561 24147     cutoff   86       119.00000  120.41829  1.19%   568 2475s
 38639 24207  120.18750   94  239  119.00000  120.41829  1.19%   567 2480s
 38818 24307  120.18071  108  246  119.00000  120.41829  1.19%   566 2487s
 38933 24383  120.13997  120  219  119.00000  120.41829  1.19%   566 2491s
 39056 24430  120.18248  139  257  119.00000  120.41829  1.19%   566 2497s
 39091 24472  120.17803  145  235  119.00000  120.41829  1.19%   565 2501s
 39255 24621  120.17598  159  232  119.00000  120.41829  1.19%   565 2507s
 39390 24659  120.15793  171  217  119.00000  120.41829  1.19%   563 2510s
 39639 24775     cutoff  196       119.00000  120.41829  1.19%   562 2516s
 39912 24863  120.03731  

## Entrada de datos: Instancia 4x200x4x1

In [52]:
datafile="4x200x4x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [53]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0xca88a036
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10706451 rows and 470320 columns (presolve time = 24s) ...
Presolve removed 10706451 rows and 470320 columns (presolve time = 29s) ...
Presolve removed 10748927 rows and 470356 columns (presolve time = 3

     0     0  110.55033    0  445  107.00000  110.55033  3.32%     -  232s
     0     0  110.54798    0  436  107.00000  110.54798  3.32%     -  232s
     0     0  110.54670    0  446  107.00000  110.54670  3.31%     -  233s
     0     0  110.54649    0  453  107.00000  110.54649  3.31%     -  233s
     0     0  110.53248    0  453  107.00000  110.53248  3.30%     -  235s
     0     0  110.52709    0  452  107.00000  110.52709  3.30%     -  236s
     0     0  110.52629    0  463  107.00000  110.52629  3.30%     -  236s
     0     0  110.52596    0  457  107.00000  110.52596  3.30%     -  236s
     0     0  110.52584    0  472  107.00000  110.52584  3.30%     -  237s
     0     0  110.51564    0  418  107.00000  110.51564  3.29%     -  238s
     0     0  110.50429    0  444  107.00000  110.50429  3.28%     -  239s
     0     0  110.50234    0  468  107.00000  110.50234  3.27%     -  240s
     0     0  110.50174    0  455  107.00000  110.50174  3.27%     -  240s
     0     0  110.50163  

  1305  1035  109.89005   77  404  108.00000  110.29575  2.13%   468  420s
  1310  1038  110.16371  125  445  108.00000  110.24014  2.07%   466  425s
  1315  1041  110.19181   30  374  108.00000  110.22007  2.06%   465  430s
  1322  1046  109.58408   92  415  108.00000  110.20391  2.04%   462  435s
  1327  1049  110.00033  181  384  108.00000  110.18416  2.02%   460  440s
  1332  1053  109.99975   61  414  108.00000  110.17418  2.01%   459  445s
  1337  1056  109.13807  111  368  108.00000  110.16619  2.01%   457  450s
  1343  1060  109.52299   22  382  108.00000  110.15464  2.00%   455  455s
  1350  1065  110.14774   10  390  108.00000  110.14774  1.99%   453  460s
  1357  1069  109.30604   11  374  108.00000  110.14201  1.98%   450  465s
  1363  1073  110.13804   64  347  108.00000  110.13804  1.98%   448  470s
  1369  1077  109.01429   17  378  108.00000  110.13629  1.98%   446  475s
  1375  1081  110.07230   57  454  108.00000  110.13542  1.98%   444  480s
  1382  1086  109.54479  

  1979  1498  109.69398    9  195  108.00000  109.69398  1.57%   854  970s
  1987  1503  109.23300   69  185  108.00000  109.69388  1.57%   851  975s
  1995  1509  109.40822   52  141  108.00000  109.69048  1.57%   847  980s
  2001  1513  109.69048   98  178  108.00000  109.69048  1.57%   845  985s
  2010  1520  109.69048  125  339  108.00000  109.69048  1.57%   897  990s
  2015  1523  109.69048   30  174  108.00000  109.69048  1.57%   895  995s
  2021  1527  109.30604   11  212  108.00000  109.69048  1.57%   892 1001s
  2025  1530  109.05067   67  199  108.00000  109.69048  1.57%   891 1005s
  2028  1535  109.69048  146  184  108.00000  109.69048  1.57%   918 1011s
  2034  1542  109.69048  148  176  108.00000  109.69048  1.57%   916 1016s
  2046  1547  109.52899  149  193  108.00000  109.68750  1.56%   913 1021s
  2058  1552  109.62968  150  232  108.00000  109.68750  1.56%   909 1025s
  2115  1593  109.56250  155  250  108.00000  109.68750  1.56%   892 1030s
  2177  1621  109.68056  

 17906 11477  109.02103  581  248  108.00000  109.66217  1.54%   464 1659s
 18099 11473  109.58015  170  464  108.00000  109.66217  1.54%   470 1676s
 18186 11567  109.39011  181  464  108.00000  109.66217  1.54%   482 1692s
 18342 11794  109.25921  192  406  108.00000  109.66217  1.54%   488 1708s
 18662 11748  109.06250  216  329  108.00000  109.66217  1.54%   491 1737s
 18752 11827  109.58333  171  388  108.00000  109.66217  1.54%   496 1752s
 18911 11892  109.50000  179  452  108.00000  109.66217  1.54%   501 1768s
 19045 11963  109.43848  196  449  108.00000  109.66217  1.54%   511 1786s
 19160 12209  109.34640  212  335  108.00000  109.66217  1.54%   515 1803s
 19468 12681  109.13411  281  372  108.00000  109.66217  1.54%   519 1823s
 20112 12670  109.34829  216  369  108.00000  109.66217  1.54%   515 1843s
 20386 12666  109.30681  222  412  108.00000  109.65294  1.53%   520 1863s
 20535 12783  109.02193  183  447  108.00000  109.65000  1.53%   529 1884s
 20805 12819  109.59478  

## Entrada de datos: Instancia 4x200x4x2

In [54]:
datafile="4x200x4x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [55]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0xf72276f1
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 12.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10703113 rows and 470020 columns (presolve time = 24s) ...
Presolve removed 10703113 rows and 470020 columns (presolve time = 29s) ...
Presolve removed 10737673 rows and 470060 columns (presolve time = 3

## Entrada de datos: Instancia 4x200x4x3

In [56]:
datafile="4x200x4x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [57]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x54d46981
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10753861 rows and 471352 columns (presolve time = 24s) ...
Presolve removed 10753861 rows and 471352 columns (presolve time = 29s) ...
Presolve removed 10792397 rows and 471392 columns (presolve time = 3

     0     0  107.74618    0  295  106.00000  107.74618  1.65%     -  202s
     0     0  107.74618    0  318  106.00000  107.74618  1.65%     -  202s
     0     0  107.74613    0  329  106.00000  107.74613  1.65%     -  202s
H    0     0                     107.0000000  107.74613  0.70%     -  203s

Cutting planes:
  Gomory: 1
  Cover: 92
  Clique: 18
  MIR: 246
  StrongCG: 130
  GUB cover: 9
  Zero half: 1
  RLT: 1
  Relax-and-lift: 4

Explored 1 nodes (29809 simplex iterations) in 204.31 seconds
Thread count was 6 (of 6 available processors)

Solution count 9: 107 106 105 ... -0

Optimal solution found (tolerance 1.00e-04)
Best objective 1.070000000000e+02, best bound 1.070000000000e+02, gap 0.0000%


## Entrada de datos: Instancia 4x200x4x4

In [58]:
datafile="4x200x4x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [59]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x90e30da3
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10726335 rows and 470728 columns (presolve time = 24s) ...
Presolve removed 10726335 rows and 470728 columns (presolve time = 29s) ...
Presolve removed 10771383 rows and 470760 columns (presolve time = 3

  6450  4062  112.30951  329  343  111.00000  112.51560  1.37%   204  357s
  7037  4591  112.14676  347  361  111.00000  112.51560  1.37%   207  363s
  7321  4836  112.09788  354  345  111.00000  112.51560  1.37%   204  365s
  7932  5308  112.28215  371  343  111.00000  112.51560  1.37%   203  372s
  8143  5480  112.05974  380  355  111.00000  112.51560  1.37%   206  375s
  8479  5842  112.27609  398  365  111.00000  112.51560  1.37%   212  382s
  8735  6041  112.27555  411  342  111.00000  112.51560  1.37%   214  386s
  8953  6215  112.27193  431  365  111.00000  112.51560  1.37%   217  390s
  9597  6864     cutoff  485       111.00000  112.51560  1.37%   219  398s
 10093  7006  112.21693  566  355  111.00000  112.51560  1.37%   219  402s
 10327  7170  112.00000  636  221  111.00000  112.51560  1.37%   226  407s
 10544  7472  112.00000  655  257  111.00000  112.51560  1.37%   230  411s
 10907  7479  112.00000  675  214  111.00000  112.51560  1.37%   236  418s
 10951  7629  112.00000  

 39248 22845  112.10776  254  452  111.00000  112.19468  1.08%   299 1226s
 39253 22849  112.08274  146  142  111.00000  112.19468  1.08%   301 1230s
 39257 22852  112.19468   61  447  111.00000  112.19468  1.08%   301 1235s
 39261 22854  112.18277   72  428  111.00000  112.18277  1.07%   301 1241s
 39265 22857  112.17103   57  445  111.00000  112.17103  1.05%   301 1245s
 39269 22860  112.00000  368  455  111.00000  112.16712  1.05%   301 1250s
 39274 22863  112.01113  227  484  111.00000  112.16108  1.05%   301 1255s
 39278 22866  112.12568  272  471  111.00000  112.15898  1.04%   301 1260s
 39282 22868  112.15493  109  491  111.00000  112.15493  1.04%   301 1265s
 39287 22872  112.04022  237  441  111.00000  112.14868  1.03%   301 1271s
 39291 22874  112.13773  207  419  111.00000  112.14430  1.03%   301 1275s
 39295 22877  112.14108  165  426  111.00000  112.14108  1.03%   301 1280s
 39300 22880  112.05171  245  495  111.00000  112.13749  1.02%   300 1285s
 39304 22883  112.00000  

 52302 28320  112.05188   88  476  111.00000  112.06296  0.96%   412 2010s
 52308 28324  112.01012  103  482  111.00000  112.06209  0.96%   412 2015s
 52314 28328  112.04193   94  460  111.00000  112.06192  0.96%   412 2020s
 52322 28334  112.01926   99  459  111.00000  112.06170  0.96%   412 2025s
 52328 28338  112.05459   78  479  111.00000  112.06087  0.96%   412 2030s
 52335 28342  112.00346  108  501  111.00000  112.06049  0.96%   412 2035s
 52340 28346  112.03397  103  494  111.00000  112.05988  0.95%   412 2040s
 52347 28350  112.00495  130  463  111.00000  112.05863  0.95%   412 2045s
 52353 28354  112.04227   86  530  111.00000  112.05810  0.95%   412 2050s
 52360 28359  112.01180  101  486  111.00000  112.05685  0.95%   412 2055s
 52366 28363  112.04114  104  458  111.00000  112.05659  0.95%   412 2060s
 52374 28368  112.01010  107  493  111.00000  112.05633  0.95%   412 2065s
 52380 28372  112.02331   91  471  111.00000  112.05629  0.95%   412 2070s
 52388 28378  112.00809  

Thread count was 6 (of 6 available processors)

Solution count 10: 111 110 110 ... -0

Time limit reached
Best objective 1.110000000000e+02, best bound 1.120000000000e+02, gap 0.9009%


## Entrada de datos: Instancia 4x200x4x5

In [60]:
datafile="4x200x4x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [61]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0xe423cf6c
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 14s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 19s) ...
Presolve removed 10729601 rows and 470812 columns (presolve time = 25s) ...
Presolve removed 10729601 rows and 470812 columns (presolve time = 30s) ...
Presolve removed 10769261 rows and 470812 columns (presolve time = 3

## Entrada de datos: Instancia 4x200x4.5x1

In [62]:
datafile="4x200x4.5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [63]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x8b15e5e3
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 14s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10746639 rows and 471584 columns (presolve time = 24s) ...
Presolve removed 10746639 rows and 471584 columns (presolve time = 29s) ...
Presolve removed 10785727 rows and 471612 columns (presolve time = 3

  1298  1078  100.76480   17  167   99.00000  100.76480  1.78%   186  255s
  1310  1086  100.76228   57  189   99.00000  100.76228  1.78%   184  260s
  1320  1093  100.54486   64  181   99.00000  100.73940  1.76%   183  265s
  1356  1110  100.73243   29  107   99.00000  100.73243  1.75%   205  270s
  1683  1328  100.72990   55   82   99.00000  100.73243  1.75%   169  275s
  2661  2263  100.71707  130   94   99.00000  100.73243  1.75%   115  280s
  4647  3079  100.36433  250   88   99.00000  100.73243  1.75%  75.7  285s
  6979  4918  100.72965   81  134   99.00000  100.73243  1.75%  62.8  290s
  8933  7243  100.46273  244  105   99.00000  100.73243  1.75%  62.3  296s
  9822  7451  100.17408   66  106   99.00000  100.73243  1.75%  61.6  302s
 10738  8755  100.06328   86  112   99.00000  100.73243  1.75%  60.9  305s
 12454 10221  100.56844  131  117   99.00000  100.73243  1.75%  60.1  310s
 13494 10222  100.42187   63  167   99.00000  100.73243  1.75%  58.1  325s
 13500 10226  100.46219  

 40171 24196  100.00000  108  279   99.00000  100.56355  1.58%   189 1125s
 40174 24198  100.00000  369  239   99.00000  100.56355  1.58%   189 1130s
 40176 24200  100.56352   94  286   99.00000  100.56352  1.58%   189 1135s
 40179 24202  100.00000  203  298   99.00000  100.56293  1.58%   189 1140s
 40182 24204  100.00000   77  243   99.00000  100.56289  1.58%   189 1146s
 40185 24206  100.00000  187  274   99.00000  100.56289  1.58%   189 1151s
 40188 24208  100.39094  102  273   99.00000  100.56289  1.58%   189 1156s
 40190 24209  100.09257  282  338   99.00000  100.56281  1.58%   189 1161s
 40192 24210  100.00000  194  278   99.00000  100.56164  1.58%   189 1165s
 40196 24213  100.00000  253  270   99.00000  100.56131  1.58%   189 1172s
 40198 24214  100.56113  114  302   99.00000  100.56113  1.58%   189 1175s
 40201 24216  100.00000  346  300   99.00000  100.56082  1.58%   189 1180s
 40204 24218  100.46598   71  258   99.00000  100.56080  1.58%   189 1185s
 40207 24220  100.00000  


Explored 53341 nodes (17637034 simplex iterations) in 2084.90 seconds
Thread count was 6 (of 6 available processors)

Solution count 9: 100 99 98 ... -0

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+02, best bound 1.000000000000e+02, gap 0.0000%


## Entrada de datos: Instancia 4x200x4.5x2

In [64]:
datafile="4x200x4.5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [65]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0xf3346d3c
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 16.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10756139 rows and 471948 columns (presolve time = 24s) ...
Presolve removed 10756139 rows and 471948 columns (presolve time = 29s) ...
Presolve removed 10797179 rows and 471976 columns (presolve time = 3

## Entrada de datos: Instancia 4x200x4.5x3

In [66]:
datafile="4x200x4.5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [67]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x8dacec60
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10735645 rows and 471552 columns (presolve time = 24s) ...
Presolve removed 10735645 rows and 471552 columns (presolve time = 29s) ...
Presolve removed 10777265 rows and 471580 columns (presolve time = 3

     0     0   99.12038    0  376   97.00000   99.12038  2.19%     -  200s
     0     0   99.12034    0  382   97.00000   99.12034  2.19%     -  200s
     0     0   99.10677    0  371   97.00000   99.10677  2.17%     -  201s
     0     0   99.10392    0  367   97.00000   99.10392  2.17%     -  201s
     0     0   99.10282    0  376   97.00000   99.10282  2.17%     -  201s
     0     0   99.10238    0  374   97.00000   99.10238  2.17%     -  202s
     0     0   99.10202    0  383   97.00000   99.10202  2.17%     -  202s
     0     0   99.10185    0  388   97.00000   99.10185  2.17%     -  202s
     0     0   99.10181    0  398   97.00000   99.10181  2.17%     -  202s
     0     0   99.08070    0  359   97.00000   99.08070  2.15%     -  203s
     0     0   99.07620    0  343   97.00000   99.07620  2.14%     -  203s
     0     0   99.07569    0  356   97.00000   99.07569  2.14%     -  204s
     0     0   99.07554    0  363   97.00000   99.07554  2.14%     -  204s
     0     0   99.06268  

## Entrada de datos: Instancia 4x200x4.5x4

In [68]:
datafile="4x200x4.5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [69]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0xb85eb7ba
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10741936 rows and 471720 columns (presolve time = 24s) ...
Presolve removed 10741936 rows and 471720 columns (presolve time = 29s) ...
Presolve removed 10785896 rows and 471748 columns (presolve time = 3

## Entrada de datos: Instancia 4x200x4.5x5

In [70]:
datafile="4x200x4.5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [71]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0xd8f140e1
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 16.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10731654 rows and 471220 columns (presolve time = 24s) ...
Presolve removed 10731654 rows and 471220 columns (presolve time = 29s) ...
Presolve removed 10771382 rows and 471252 columns (presolve time = 3

  1399   660  105.04161   28  243  104.00000  105.45680  1.40%   244  295s
  1405   664  105.44783   14  228  104.00000  105.44783  1.39%   243  301s
  1411   668  105.09552   41  243  104.00000  105.44501  1.39%   242  305s
  1417   672  105.44369   15  281  104.00000  105.44369  1.39%   241  310s
  1423   676  105.44203   15  259  104.00000  105.44203  1.39%   240  315s
  1430   680  105.43979   25  307  104.00000  105.43979  1.38%   238  320s
  1437   685  105.40871   73  263  104.00000  105.43747  1.38%   237  325s
  1444   690  105.42606   90  320  104.00000  105.43684  1.38%   236  330s
  1450   694  105.03275   37  357  104.00000  105.43279  1.38%   235  335s
  1456   698  105.14592   37  380  104.00000  105.43205  1.38%   234  340s
  1464   704  105.17545   32  177  104.00000  105.43198  1.38%   290  347s
  1469   707  105.05655   50  162  104.00000  105.43198  1.38%   289  350s
  1474   711  105.30132   14  356  104.00000  105.42984  1.37%   288  355s
  1481   715  105.36498  

 11379  5040     cutoff  112       104.00000  105.00000  0.96%   380  875s
 11749  5108     cutoff   94       104.00000  105.00000  0.96%   386  881s
 12179  5141  105.00000  108  246  104.00000  105.00000  0.96%   391  888s
 12348  5205  105.00000  118  179  104.00000  105.00000  0.96%   400  894s
 12761  5206  105.00000  117  177  104.00000  105.00000  0.96%   405  900s
 12772  5213  105.00000  114   81  104.00000  105.00000  0.96%   405  905s
 12782  5220  105.00000  103   69  104.00000  105.00000  0.96%   404  910s
 12846  5274  105.00000   87   93  104.00000  105.00000  0.96%   407  915s
 13084  5614  105.00000  102   79  104.00000  105.00000  0.96%   402  923s
 13346  6180  105.00000  119  127  104.00000  105.00000  0.96%   399  929s
 14217  6237  105.00000   92   83  104.00000  105.00000  0.96%   383  934s
 14733  6392     cutoff  139       104.00000  105.00000  0.96%   376  938s
 15199  6359     cutoff  104       104.00000  105.00000  0.96%   370  941s
 15399  7177  105.00000  

## Entrada de datos: Instancia 4x200x5x1

In [72]:
datafile="4x200x5x1.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [73]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x6ffdc199
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 15.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10788324 rows and 472844 columns (presolve time = 24s) ...
Presolve removed 10788324 rows and 472844 columns (presolve time = 29s) ...
Presolve removed 10828092 rows and 472868 columns (presolve time = 3

 23477 15032     cutoff  125        93.00000   94.00000  1.08%   155  290s
 25027 15597   94.00000   80  118   93.00000   94.00000  1.08%   159  296s
 25843 15598   94.00000  140   79   93.00000   94.00000  1.08%   163  320s
 25856 15607   94.00000  105  151   93.00000   94.00000  1.08%   163  325s
 25860 15609   94.00000   52  182   93.00000   94.00000  1.08%   163  332s
 25869 15610   94.00000   41  138   93.00000   94.00000  1.08%   164  335s
 25906 15628   94.00000   51  142   93.00000   94.00000  1.08%   164  340s
 26195 16330   94.00000   64   88   93.00000   94.00000  1.08%   163  346s
 28614 16374     cutoff  115        93.00000   94.00000  1.08%   155  350s
 29342 16615   94.00000  101  136   93.00000   94.00000  1.08%   157  355s
 30498 16778   94.00000   94  160   93.00000   94.00000  1.08%   159  361s
 31572 16925   94.00000   72  145   93.00000   94.00000  1.08%   160  366s
 32473 16863 infeasible   89        93.00000   94.00000  1.08%   161  370s
 33585 17100   94.00000  

## Entrada de datos: Instancia 4x200x5x2

In [74]:
datafile="4x200x5x2.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [75]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0xc47b7514
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10772496 rows and 472552 columns (presolve time = 24s) ...
Presolve removed 10772496 rows and 472552 columns (presolve time = 29s) ...
Presolve removed 10811532 rows and 472576 columns (presolve time = 3

  1525  1286   94.05896   31  152   92.00000   94.05896  2.24%   120  260s
  1537  1295   94.05896   86  106   92.00000   94.05896  2.24%   133  265s
  1548  1302   94.00000  101   86   92.00000   94.00000  2.17%   132  270s
  1557  1309   94.00000   46  114   92.00000   94.00000  2.17%   149  275s
  1568  1316   93.27001   48   91   92.00000   94.00000  2.17%   148  280s
  1578  1323   94.00000    9   95   92.00000   94.00000  2.17%   147  285s
  1639  1388   94.00000   71   83   92.00000   94.00000  2.17%   160  291s
  2214  2010   94.00000  136   74   92.00000   94.00000  2.17%   135  298s
  2538  2528   94.00000  155   75   92.00000   94.00000  2.17%   125  301s
  4468  3782   93.07901  138  108   92.00000   94.00000  2.17%  82.8  305s
  6838  4840   93.99661  129   95   92.00000   94.00000  2.17%  61.4  313s
  7156  5153     cutoff  155        92.00000   94.00000  2.17%  60.0  319s
  7500  6873   93.76518  156   94   92.00000   94.00000  2.17%  58.5  321s
 11553 10304   93.69830  

## Entrada de datos: Instancia 4x200x5x3

In [76]:
datafile="4x200x5x3.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [77]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0xd2e7627b
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10776309 rows and 472504 columns (presolve time = 24s) ...
Presolve removed 10776309 rows and 472504 columns (presolve time = 29s) ...
Presolve removed 10817249 rows and 472528 columns (presolve time = 3

## Entrada de datos: Instancia 4x200x5x4

In [78]:
datafile="4x200x5x4.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [79]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])

ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x7ecd0466
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 14.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10744024 rows and 472132 columns (presolve time = 24s) ...
Presolve removed 10744024 rows and 472132 columns (presolve time = 29s) ...
Presolve removed 10786060 rows and 472156 columns (presolve time = 3

  2367  1921   93.00000   32  101   92.00000   93.00000  1.09%  65.7  285s
  2378  1928   93.00000   60   58   92.00000   93.00000  1.09%  65.4  290s
  2415  1960   93.00000   41   76   92.00000   93.00000  1.09%  74.1  296s
  2722  2889   93.00000   56   72   92.00000   93.00000  1.09%  68.0  300s
  6249  3934   93.00000  112   87   92.00000   93.00000  1.09%  45.0  305s
  7633  3650   93.00000  131   63   92.00000   93.00000  1.09%  42.6  315s
 11399  5867   93.00000  128   71   92.00000   93.00000  1.09%  48.9  320s
 12692  6207   93.00000  144  126   92.00000   93.00000  1.09%  51.0  331s
 12701  6213   93.00000  137   37   92.00000   93.00000  1.09%  50.9  335s
 12711  6220   93.00000  142   55   92.00000   93.00000  1.09%  50.9  341s
 12718  6229   93.00000   50   65   92.00000   93.00000  1.09%  52.8  345s
 12860  6396   93.00000   61   69   92.00000   93.00000  1.09%  52.5  350s
 13109  7602   93.00000  108   67   92.00000   93.00000  1.09%  52.1  359s
 14603  7403 infeasible  

 126400 56235     cutoff  203        92.00000   93.00000  1.09%   127 1019s
 127196 56930   93.00000  137  106   92.00000   93.00000  1.09%   128 1024s
 128545 57330   93.00000  282  128   92.00000   93.00000  1.09%   128 1029s
 129248 57905   93.00000  177   73   92.00000   93.00000  1.09%   129 1036s
 130086 58244   93.00000  260  140   92.00000   93.00000  1.09%   129 1044s
 130515 58552     cutoff  274        92.00000   93.00000  1.09%   129 1058s
 130971 59964   93.00000  201   60   92.00000   93.00000  1.09%   129 1067s
 133157 60591     cutoff  170        92.00000   93.00000  1.09%   130 1073s
 134316 61022   93.00000  243  106   92.00000   93.00000  1.09%   131 1078s
 135067 61951   93.00000  184   94   92.00000   93.00000  1.09%   131 1084s
 136730 62104     cutoff  192        92.00000   93.00000  1.09%   131 1096s
 137011 62566   93.00000  194   90   92.00000   93.00000  1.09%   131 1101s
 137865 63114   93.00000  120  105   92.00000   93.00000  1.09%   132 1107s
 138708 6362

## Entrada de datos: Instancia 4x200x5x5

In [80]:
datafile="4x200x5x5.csv"
data=pd.DataFrame(pd.read_csv(datafile,sep=";"))

data["JOB"]="J"+data["JOB"].map(str)
dictJobs={i:[data.set_index('JOB').to_dict()["Deadline"][i],data.set_index('JOB').to_dict()["PT"][i]] for i in data.set_index('JOB').to_dict()["Deadline"].keys()}

dictJobs=dict(sorted(dictJobs.items(), key=lambda item: item[1]))
ordDict={}
cont=0
for i in dictJobs:
    ordDict[cont]=[i,dictJobs[i][0],dictJobs[i][1]]
    cont+=1
len(dictJobs)

200

In [81]:
p = []
d = []
for i in ordDict.keys():
    p.append(ordDict[i][2])
    d.append(ordDict[i][1])
    
ith = len(ordDict)
jth = 4
N = range(ith)
M = range(jth)
BIGM = 10000000
HT = 24
t =[0,24,48,72,96,168,192,216,240,264,336,360,384,408,432,504,528,552,576,600,672,696,720,744,768,840,864,888,912,936,1008,1032,1056,1080,1104,1176,1200,1224,1248,1272,1344,1368,1392,1416,1440,1512,1536,1560,1584,1608,1680,1704,1728,1752,1776,1848,1872,1896,1920,1944,2016,2040,2064,2088,2112,2184,2208,2232,2256,2280,2352,2376,2400,2424,2448,2520,2544,2568,2592,2616,2688,2712,2736,2760,2784,2856,2880,2904,2928,2952,3024,3048,3072,3096,3120,3192,3216,3240,3264,3288,3360,3384,3408,3432,3456,3528,3552,3576,3600,3624,3696,3720,3744,3768,3792,3864,3888,3912,3936,3960,4032,4056,4080,4104,4128,4200,4224,4248,4272,4296,4368,4392,4416,4440,4464,4536,4560,4584,4608,4632,4704,4728,4752,4776,4800,4872,4896,4920,4944,4968,5040,5064,5088,5112,5136,5208,5232,5256,5280,5304,5376,5400,5424,5448,5472,5544,5568,5592,5616,5640,5712,5736,5760,5784,5808,5880,5904,5928,5952,5976,6048,6072,6096,6120,6144,6216,6240,6264,6288,6312,6384,6408,6432,6456,6480,6552,6576,6600,6624,6648,6720,6744,6768,6792,6816,6888,6912,6936,6960,6984,7056,7080,7104,7128,7152,7224,7248,7272,7296,7320,7392,7416,7440,7464,7488,7560,7584,7608,7632,7656,7728,7752,7776,7800,7824,7896,7920,7944,7968,7992,8064,8088,8112,8136,8160,8232,8256,8280,8304,8328,8400,8424,8448,8472,8496,8568,8592,8616,8640,8664,8736,8760,8784,8808,8832,8904,8928,8952,8976,9000,9072,9096,9120,9144,9168,9240,9264,9288,9312,9336,9408,9432,9456,9480,9504,9576,9600,9624,9648,9672,9744,9768,9792,9816,9840,9912,9936,9960,9984,10008,10080,10104,10128,10152,10176,10248,10272,10296,10320,10344,10416,10440,10464,10488,10512,10584,10608,10632,10656,10680,10752,10776,10800,10824,10848,10920,10944,10968,10992,11016,11088,11112,11136,11160,11184,11256,11280,11304,11328,11352,11424,11448,11472,11496,11520,11592,11616,11640,11664,11688,11760,11784,11808,11832,11856,11928,11952,11976,12000,12024,12096,12120,12144,12168,12192,12264,12288,12312,12336,12360,12432,12456,12480,12504,12528,12600,12624,12648,12672,12696,12768,12792,12816,12840,12864,12936,12960,12984,13008,13032,13104,13128,13152,13176,13200,13272,13296,13320,13344,13368,13440,13464,13488,13512,13536,13608,13632,13656,13680,13704,13776,13800,13824,13848,13872,13944,13968,13992,14016,14040,14112,14136,14160,14184,14208,14280,14304,14328,14352,14376,14448,14472,14496,14520,14544,14616,14640,14664,14688,14712,14784,14808,14832,14856,14880,14952,14976,15000,15024,15048,15120,15144,15168,15192,15216,15288,15312,15336,15360,15384,15456,15480,15504,15528,15552,15624,15648,15672,15696,15720,15792,15816,15840,15864,15888,15960,15984,16008,16032,16056,16128,16152,16176,16200,16224,16296,16320,16344,16368,16392,16464,16488,16512,16536,16560,16632,16656,16680,16704,16728,16800,16824,16848,16872,16896,16968]

WT = 11

indices=[(i,j,k) for i in N for j in M for k in N]

#Inicializar modelo
mdl = grb.Model("modelo")

#x = mdl.addVars(indices, vtype=grb.BINARY)
x = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='x')
y = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')
MAX = mdl.addVars(indices, vtype=grb.GRB.BINARY, name='y')

mdl.Params.timeLimit=3600.0
mdl.setObjective(sum(x[(i,j,k)]+y[(i,j,k)] for i in N for j in M for k in N))

mdl.addConstrs((sum(y[(i,j,k)] for i in N) <= 1 for k in N for j in M),name="Primera")
mdl.addConstrs((len(range(i,ith))*(1-y[(i,j,k)]) >=sum(x[(z,j,k)] for z in range(i,ith)) for k in N for j in M for i in N), name='C')
mdl.addConstrs((sum(x[(i,j,k)]+y[(i,j,k)] for j in M for k in N) <= 1 for i in N), name='B')

mdl.addConstrs((sum((x[(i,j,k)]+y[(i,j,k)])*p[i] for i in N) <= HT for k in N for j in M) , name='A')

mdl.addConstrs((sum(x[(i,j,k)]*p[i] for i in N) <= WT  for k in N for j in M), name='D')

for i in N:
    for j in M:
        for k in range(i+1):      
            mdl.addConstr(sum((x[(z,j,k)]+y[(z,j,k)])*p[z] for z in range(i+1))+(t[k]*MAX[(i,j,k)]) <= d[i])
            
for i in N:
    for j in M:
        for k in range(i+1):
            for z in range(i+1):
                mdl.addConstr(x[(z,j,k)]+y[(z,j,k)] <=MAX[(i,j,k)])
                
mdl.ModelSense = grb.GRB.MAXIMIZE
mdl.optimize()

Changed value of parameter timeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 10989800 rows, 480000 columns and 71013600 nonzeros
Model fingerprint: 0x940cb400
Variable types: 0 continuous, 480000 integer (480000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 13.0000000
Presolve removed 0 rows and 80400 columns (presolve time = 13s) ...
Presolve removed 0 rows and 80400 columns (presolve time = 18s) ...
Presolve removed 10757293 rows and 472240 columns (presolve time = 24s) ...
Presolve removed 10757293 rows and 472240 columns (presolve time = 29s) ...
Presolve removed 10798505 rows and 472264 columns (presolve time = 3

     0     0   94.63479    0  239   93.00000   94.63479  1.76%     -  216s
     0     0   94.63479    0  154   93.00000   94.63479  1.76%     -  218s
     0     0   94.63479    0  136   93.00000   94.63479  1.76%     -  218s
     0     2   94.63479    0  128   93.00000   94.63479  1.76%     -  219s
    13    20   94.63479    4  258   93.00000   94.63479  1.76%   325  220s
   177   272   94.61677   22  148   93.00000   94.63479  1.76%   142  240s
   568   627   94.37115   60  114   93.00000   94.63479  1.76%  82.1  248s
   751   756   94.05470   77  108   93.00000   94.63479  1.76%  76.4  264s
   928   884     cutoff   27        93.00000   94.63479  1.76%  85.7  283s
  1144  1288   94.63479   22  272   93.00000   94.63479  1.76%  91.1  290s
  1730  1289   94.31240   38  136   93.00000   94.63479  1.76%   103  301s
  1737  1294   94.17862   21  189   93.00000   94.17862  1.27%   103  305s
  1744  1298   94.13408   48  277   93.00000   94.17862  1.27%   102  310s
  1751  1303   94.17862  

 39203 19908   94.15477   58  335   93.00000   94.16375  1.25%   230 1436s
 39316 19921   94.13845   62  328   93.00000   94.16375  1.25%   230 1456s
 39577 20140   94.00000  112  236   93.00000   94.16375  1.25%   231 1463s
 39756 20212   94.00000  138  225   93.00000   94.16375  1.25%   231 1466s
 40121 20387   94.00000  215  222   93.00000   94.16375  1.25%   231 1472s
 40302 20454   94.00000  239  217   93.00000   94.16375  1.25%   232 1475s
 40694 20543   94.00000   84  223   93.00000   94.16375  1.25%   233 1483s
 40790 20549   94.00000  121  176   93.00000   94.16375  1.25%   233 1487s
 40883 20545   94.00000  136  238   93.00000   94.16375  1.25%   234 1491s
 40940 20599   94.00000  145  240   93.00000   94.16375  1.25%   235 1495s
 41134 20627   94.01560   57  392   93.00000   94.16371  1.25%   237 1501s
 41343 20739     cutoff   72        93.00000   94.16254  1.25%   239 1508s
 41470 20791   94.00000   75  309   93.00000   94.16254  1.25%   240 1511s
 41581 20881   94.00000  